![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Clinical Entity Coding with Pretrained Resolver Models 

In [0]:
import os
import json
import string
import numpy as np
import pandas as pd


import sparknlp
import sparknlp_jsl
from sparknlp.base import *
from sparknlp.util import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.pretrained import ResourceDownloader

from pyspark.sql import functions as F
from pyspark.ml import Pipeline, PipelineModel

pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', 100)  
pd.set_option('display.expand_frame_repr', False)


print('sparknlp_jsl.version : ',sparknlp_jsl.version())

spark

sparknlp_jsl.version : 3.0.1
Out[1]:

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.0.1 
 Master 
 spark://10.164.250.91:7077 
 AppName 
 Databricks Shell

# Clinical Resolvers

## Entity Resolvers for ICD-10

A common NLP problem in biomedical aplications is to identify the presence of clinical entities in a given text. This clinical entities could be diseases, symptoms, drugs, results of clinical investigations or others.

Other than providing the code in the "result" field it provides more metadata about the matching process:

- target_text -> Text to resolve
- resolved_text -> Best match text
- confidence -> Relative confidence for the top match (distance to probability)
- confidence_ratio -> Relative confidence for the top match. TopMatchConfidence / SecondMatchConfidence
- alternative_codes -> List of other plausible codes (in the KNN neighborhood)
- alternative_confidence_ratios -> Rest of confidence ratios
- all_k_results -> All resolved codes for metrics calculation purposes
- sentence -> SentenceId
- chunk -> ChunkId

### Clinical NER Pipeline creation

In [0]:
# Annotator that transforms a text column from dataframe into an Annotation ready for NLP

documentAssembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

# Sentence Detector DL annotator, processes various sentences per line
sentenceDetectorDL = SentenceDetectorDLModel.pretrained("sentence_detector_dl_healthcare", "en", 'clinical/models') \
  .setInputCols(["document"]) \
  .setOutputCol("sentence")

# Tokenizer splits words in a relevant format for NLP

tokenizer = Tokenizer()\
  .setInputCols(["sentence"])\
  .setOutputCol("raw_token")\

# Tokenizer splits words in a relevant format for NLP

stopwords = StopWordsCleaner()\
  .setInputCols(["raw_token"])\
  .setOutputCol("token")
  

sentence_detector_dl_healthcare download started this may take some time.
Approximate size to download 363.9 KB
[ | ][OK!]

The fourth annotator in the pipeline is "WordEmbeddingsModel". We will download a pretrained model available from "clinical/models" named "embeddings_clinical".

When running this cell your are advised to be patient.

First time you call this pretrained model it needs to be downloaded in your local.

The model size is about will download the embeddings_clinical corpus it takes a while.

The size is about 1.7Gb and will be saved typically in your home folder as

`~HOMEFOLDER/cached_models/ embeddings_clinical_en_2.0.2_2.4_1558454742956`

Next times you call it the model is loaded from your cached copy but even in that case it needs to be indexed each time so expect waiting up to 5 minutes (depending on your machine)

In [0]:
# WordEmbeddingsModel pretrained "embeddings_clinical" includes a model of 1.6Gb that needs to be downloaded

word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
  .setInputCols(["sentence", "token"])\
  .setOutputCol("embeddings")
  

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]

The fifth and final annotator in our NER pipeline is the pretrained `ner_clinical` NerDLModel avaliable from "clinical/models". It requires as input the "sentence", "token" and "embeddings" (clinical embeddings pretrained model) and will classify each token in four categories:

- `PROBLEM`: for patient problems

- `TEST`: for tests, labs, etc.

- `TREATMENT`: for treatments, medicines, etc.

- `OTHER`: for the rest of tokens.

In order to split those identified NER that are consecutive, the B prefix (as B-PROBLEM) will be used at the first token of each NER. The I prefix (as I-PROBLEM) will be used for the rest of tokens inside the NER.

In [0]:
# Named Entity Recognition for clinical concepts.

clinical_ner = MedicalNerModel.pretrained("ner_clinical", "en", "clinical/models") \
  .setInputCols(["sentence", "token", "embeddings"]) \
  .setOutputCol("ner")

ner_converter = NerConverter() \
    .setInputCols(["sentence", "token", "ner"]) \
    .setOutputCol("ner_chunk")


ner_clinical download started this may take some time.
Approximate size to download 13.9 MB
[ | ][ / ][OK!]

### Define the NER pipeline

Now we will define the actual pipeline that puts together the annotators we have created.

In [0]:
# Build up the pipeline

pipeline_ner = Pipeline(
    stages = [
    documentAssembler,
    sentenceDetectorDL,
    tokenizer,
    stopwords,
    word_embeddings,
    clinical_ner,
    ner_converter
  ])

### Create a SparkDataFrame with the content

Now we will create a sample Spark dataframe with our clinical note example.

In this example we are working over a unique clinical note. In production environments a table with several of those clinical notes could be distributed in a cluster and be run in large scale systems.

In [0]:

clinical_note = (
    'A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years '
    'prior to presentation and subsequent type two diabetes mellitus (T2DM), one prior '
    'episode of HTG-induced pancreatitis three years prior to presentation, associated '
    'with an acute hepatitis, and obesity with a body mass index (BMI) of 33.5 kg/m2, '
    'presented with a one-week history of polyuria, polydipsia, poor appetite, and vomiting. '
    'Two weeks prior to presentation, she was treated with a five-day course of amoxicillin '
    'for a respiratory tract infection. She was on metformin, glipizide, and dapagliflozin '
    'for T2DM and atorvastatin and gemfibrozil for HTG. She had been on dapagliflozin for six months '
    'at the time of presentation. Physical examination on presentation was significant for dry oral mucosa; '
    'significantly, her abdominal examination was benign with no tenderness, guarding, or rigidity. Pertinent '
    'laboratory findings on admission were: serum glucose 111 mg/dl, bicarbonate 18 mmol/l, anion gap 20, '
    'creatinine 0.4 mg/dL, triglycerides 508 mg/dL, total cholesterol 122 mg/dL, glycated hemoglobin (HbA1c) '
    '10%, and venous pH 7.27. Serum lipase was normal at 43 U/L. Serum acetone levels could not be assessed '
    'as blood samples kept hemolyzing due to significant lipemia. The patient was initially admitted for '
    'starvation ketosis, as she reported poor oral intake for three days prior to admission. However, '
    'serum chemistry obtained six hours after presentation revealed her glucose was 186 mg/dL, the anion gap '
    'was still elevated at 21, serum bicarbonate was 16 mmol/L, triglyceride level peaked at 2050 mg/dL, and '
    'lipase was 52 U/L. The β-hydroxybutyrate level was obtained and found to be elevated at 5.29 mmol/L - '
    'the original sample was centrifuged and the chylomicron layer removed prior to analysis due to '
    'interference from turbidity caused by lipemia again. The patient was treated with an insulin drip '
    'for euDKA and HTG with a reduction in the anion gap to 13 and triglycerides to 1400 mg/dL, within '
    '24 hours. Her euDKA was thought to be precipitated by her respiratory tract infection in the setting '
    'of SGLT2 inhibitor use. The patient was seen by the endocrinology service and she was discharged on '
    '40 units of insulin glargine at night, 12 units of insulin lispro with meals, and metformin 1000 mg '
    'two times a day. It was determined that all SGLT2 inhibitors should be discontinued indefinitely. She '
    'had close follow-up with endocrinology post discharge.'
)

data_ner = spark.createDataFrame([[clinical_note]]).toDF("text")

In [0]:
data_ner.show(truncate = 100)


+----------------------------------------------------------------------------------------------------+
 text|
+----------------------------------------------------------------------------------------------------+
A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior ...|
+----------------------------------------------------------------------------------------------------+

### Transform / annotate the clinical note using the model.

In order to process the data with the new created model we have two options.

The first one would be to use the model to transform our clinical note by the command:

`output = model_ner.transform(data_ner)`

That would save in a Spakr DataFrame (output) the resuls of running the model over the clinical note.

However for small tests like this or for real-time request a LightPipelines is a simpler way of managing the data. It will return a dictionary (instead of a Spark DataFrame) with the results of the transformation

We will create a light_pipeline_ner using our model_ner and then will annotate the clinical_note using this light_pipeline.

In [0]:
model = pipeline_ner.fit(data_ner)

light_pipeline = LightPipeline(model)
light_data = light_pipeline.annotate(clinical_note)

Now we have a dictionaty (light_data_ner) that contains the results of running the NER pipeline over our clinical note.

It contains the original document:

In [0]:
light_data['document'][0][0:100]


Out[10]: 'A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to '

In [0]:
print("Number of sentences: {}".format(len(light_data['sentence'])))
print("")
for i in range(5):
    print("Sentence {}: {}".format(i, light_data['sentence'][i]))

Number of sentences: 16

Sentence 0: A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus (T2DM), one prior episode of HTG-induced pancreatitis three years prior to presentation, associated with an acute hepatitis, and obesity with a body mass index (BMI) of 33.5 kg/m2, presented with a one-week history of polyuria, polydipsia, poor appetite, and vomiting.
Sentence 1: Two weeks prior to presentation, she was treated with a five-day course of amoxicillin for a respiratory tract infection.
Sentence 2: She was on metformin, glipizide, and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG.
Sentence 3: She had been on dapagliflozin for six months at the time of presentation.
Sentence 4: Physical examination on presentation was significant for dry oral mucosa; significantly, her abdominal examination was benign with no tenderness, guarding, or rigidity.

In [0]:
print("Number of tokens: {}".format(len(light_data['token'])))
print("")
for i in range(25):
    print("Token {}: {} ({})".format(i, light_data['token'][i], light_data['ner'][i]))
print("...")

Number of tokens: 299

Token 0: 28-year-old (O)
Token 1: female (O)
Token 2: history (O)
Token 3: gestational (B-PROBLEM)
Token 4: diabetes (I-PROBLEM)
Token 5: mellitus (I-PROBLEM)
Token 6: diagnosed (O)
Token 7: eight (O)
Token 8: years (O)
Token 9: prior (O)
Token 10: presentation (O)
Token 11: subsequent (O)
Token 12: type (B-PROBLEM)
Token 13: two (I-PROBLEM)
Token 14: diabetes (I-PROBLEM)
Token 15: mellitus (I-PROBLEM)
Token 16: ( (O)
Token 17: T2DM (B-PROBLEM)
Token 18: ), (O)
Token 19: one (B-PROBLEM)
Token 20: prior (I-PROBLEM)
Token 21: episode (I-PROBLEM)
Token 22: HTG-induced (I-PROBLEM)
Token 23: pancreatitis (I-PROBLEM)
Token 24: three (O)
...

Lets apply some HTML formating to see the results of the pipeline in a nicer layout:

In [0]:
from sparknlp_display import NerVisualizer

light_result = light_pipeline.fullAnnotate(clinical_note)

visualiser = NerVisualizer()

# Change color of an entity label
visualiser.set_label_colors({'PROBLEM':'#008080', 'TEST':'#800080', 'TREATMENT':'#808080'})
vis = visualiser.display(light_result[0], label_col='ner_chunk', return_html=True)

# Set label filter
# vis = visualiser.display(light_result, label_col='ner_chunk', document_col='document',
                   #labels=['PROBLEM','TEST'])
  
displayHTML(vis)

A 28-year-old female with a history of gestational diabetes mellitus PROBLEM diagnosed eight years prior to presentation and subsequent type two diabetes mellitus PROBLEM ( T2DM PROBLEM ), one prior episode of HTG-induced pancreatitis PROBLEM three years prior to presentation, associated with an acute hepatitis PROBLEM , and obesity with a body mass index PROBLEM (BMI) of 33.5 kg/m2, presented with a one-week history of polyuria PROBLEM , polydipsia PROBLEM , poor appetite PROBLEM , and vomiting PROBLEM . Two weeks prior to presentation, she was treated with a five-day course of amoxicillin for a respiratory tract infection PROBLEM . She was on metformin TREATMENT , glipizide TREATMENT , and dapagliflozin TREATMENT for T2DM PROBLEM and atorvastatin and gemfibrozil TREATMENT for HTG PROBLEM . She had been on dapagliflozin TREATMENT for six months at the time of presentation. Physical examination TEST on presentation was significant for dry oral mucosa PROBLEM ; significantly, her abdominal examination TEST was benign with no tenderness PROBLEM , guarding PROBLEM , or rigidity PROBLEM . Pertinent laboratory findings on admission were: serum glucose TEST 111 mg/dl, bicarbonate TEST 18 mmol/l, anion gap TEST 20, creatinine TEST 0.4 mg/dL, triglycerides TEST 508 mg/dL, total cholesterol TEST 122 mg/dL, glycated hemoglobin TEST ( HbA1c TEST ) 10%, and venous pH TEST 7.27. Serum lipase TEST was normal at 43 U/L TEST . Serum acetone levels TEST could not be assessed as blood samples TEST kept hemolyzing due to significant lipemia PROBLEM . The patient was initially admitted for starvation ketosis PROBLEM , as she reported poor oral intake PROBLEM for three days prior to admission. However, serum chemistry TEST obtained six hours after presentation revealed her glucose TEST was 186 mg/dL, the anion gap TEST was still elevated PROBLEM at 21, serum bicarbonate TEST was 16 mmol/L, triglyceride level TEST peaked at 2050 mg/dL, and lipase TEST was 52 U/L TEST . The β-hydroxybutyrate level TEST was obtained and found to be elevated PROBLEM at 5.29 mmol/L - the original sample was centrifuged and the chylomicron layer removed prior to analysis TEST due to interference from turbidity PROBLEM caused by lipemia PROBLEM again. The patient was treated with an insulin drip for euDKA and HTG with a reduction in the anion gap TREATMENT to 13 and triglycerides TEST to 1400 mg/dL, within 24 hours. Her euDKA was thought to be precipitated by her respiratory tract infection PROBLEM in the setting of SGLT2 inhibitor TREATMENT use. The patient was seen by the endocrinology service and she was discharged on 40 units of insulin glargine TREATMENT at night, 12 units of insulin lispro with meals TREATMENT , and metformin TREATMENT 1000 mg two times a day. It was determined that all SGLT2 inhibitors TREATMENT should be discontinued indefinitely. She had close follow-up with endocrinology post discharge.

##  ICD10 background info

ICD-10-CM vs. ICD-10-PCS

With the transition to ICD-10, in the United States, ICD-9 codes are segmented into ICD-10-CM and ICD-10-PCS codes. **The "CM" in ICD-10-CM codes stands for clinical modification**; ICD-10-CM codes were developed by the Centers for Disease Control and Prevention in conjunction with the National Center for Health Statistics (NCHS), for outpatient medical coding and reporting in the United States, as published by the World Health Organization (WHO).

**The "PCS" in ICD-10-PCS codes stands for the procedural classification system**. ICD-10-PCS is a completely separate medical coding system from ICD-10-CM, containing an additional 87,000 codes for use ONLY in United States inpatient, hospital settings. The procedure classification system (ICD-10-PCS) was developed by the Centers for Medicare and Medicaid Services (CMS) in conjunction with 3M Health Information Management (HIM).

ICD-10-CM codes add increased specificity to their ICD-9 predecessors, growing to five times the number of codes as the present system; a total of 68,000 clinical modification diagnosis codes. ICD-10-CM codes provide the ability to track and reveal more information about the quality of healthcare, allowing healthcare providers to better understand medical complications, better design treatment and care, and better comprehend and determine the outcome of care.

ICD-10-PCS is used only for inpatient, hospital settings in the United States, and is meant to replace volume 3 of ICD-9 for facility reporting of inpatient procedures. Due to the rapid and constant state of flux in medical procedures and technology, ICD-10-PCS was developed to accommodate the changing landscape. Common procedures, lab tests, and educational sessions that are not unique to the inpatient, hospital setting have been omitted from ICD-10-PCS.

ICD-10 is confusing enough when you’re trying to digest the differences between ICD-9 and ICD-10, but there are also different types of ICD-10 codes that providers should be aware of.


Primary difference between ICD-10-CM and ICD-10-PCS

When most people talk about ICD-10, they are referring to ICD-10CM. This is the code set for diagnosis coding and is used for all healthcare settings in the United States. ICD-10PCS, on the other hand, is used in hospital inpatient settings for inpatient procedure coding.

ICD-10-CM breakdown

- Approximately 68,000 codes
- 3–7 alphanumeric characters
- Facilitates timely processing of claims


ICD-10-PCS breakdown

- Will replace ICD-9-CM for hospital inpatient use only. 
- ICD-10-PCS will not replace CPT codes used by physicians. According to HealthCare Information Management, Inc. (HCIM), “Its only intention is to identify inpatient facility services in a way not directly related to physician work, but directed towards allocation of hospital services.”

- 7 alphanumeric characters

ICD-10-PCS is very different from ICD-9-CM procedure coding due to its ability to be more specific and accurate. “This becomes increasingly important when assessing and tracking the quality of medical processes and outcomes, and compiling statistics that are valuable tools for research,” according to HCIM.

# ChunkEntityResolver Models

## ICD10CM Resolver

**ICD10 Coding Pipeline Creation**

A common NLP problem in biomedical aplications is to identify the presence of clinical entities in a given text. This clinical entities could be diseases, symptoms, drugs, results of clinical investigations or others.

Other than providing the code in the "result" field it provides more metadata about the matching process:

- target_text -> Text to resolve
- resolved_text -> Best match text
- confidence -> Relative confidence for the top match (distance to probability)
- confidence_ratio -> Relative confidence for the top match. TopMatchConfidence / SecondMatchConfidence
- alternative_codes -> List of other plausible codes (in the KNN neighborhood)
- alternative_confidence_ratios -> Rest of confidence ratios
- all_k_results -> All resolved codes for metrics calculation purposes
- sentence -> SentenceId
- chunk -> ChunkId

We will now create a new pipeline that from each of these problems will try to assign an ICD10 base on the content, the wordembeddings and some pretrained models for ICD10 annotation.

The architecture of this new pipeline will be as follows:

- DocumentAssembler (text -> document)

- SentenceDetector (document -> sentence)

- Tokenizer (sentence -> token)

- WordEmbeddingsModel ([sentence, token] -> embeddings)

- NerDLModel ([sentence, token, embeddings] -> ner)

- NerConverter (["sentence, token, ner] -> ner_chunk

- ChunkTokenizer (ner_chunk -> ner_chunk_tokenized)

- ICD10CMEntityResolverModel ([ner_chunk_tokenized, embeddings] -> resolution)

- ICD10PCSEntityResolverModel ([ner_chunk_tokenized, embeddings] -> resolution)

So from a text we end having a list of Named Entities (ner_chunk) and their ICD10 codes (resolution)

Most of the annotators in this pipeline have been already created for the previous pipeline, but we need to create four additional annotators: NerConverter, ChunkEmbeddigns, EntityResolverModel for ICD10CM and EntityResolverModel for ICD10PCS.

Now we define the new pipeline

In [0]:
# Named Entity Recognition concepts parser, transforms entities into CHUNKS (required for next step: assertion status)

ner_converter = NerConverterInternal() \
  .setInputCols(["sentence", "token", "ner"]) \
  .setOutputCol("ner_chunk")\
  .setWhiteList(['PROBLEM'])\
  .setPreservePosition(False)

chunk_embeddings = ChunkEmbeddings()\
  .setInputCols("ner_chunk", "embeddings")\
  .setOutputCol("chunk_embeddings")

# ICD resolution model
icd10cm_resolution = ChunkEntityResolverModel.pretrained("chunkresolve_icd10cm_clinical", "en", "clinical/models") \
  .setInputCols(["token", "chunk_embeddings"]) \
  .setOutputCol("icd10cm_code") \
  .setDistanceFunction("COSINE") \
  .setNeighbours(5)
# .setDistanceFunction("EUCLIDEAN")

chunkresolve_icd10cm_clinical download started this may take some time.
Approximate size to download 166.2 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][OK!]

`setPreservePosition(True)` takes exactly the original indices (under some tokenization conditions it might include some undesires chars like `")","]"...)`

`setPreservePosition(False)` takes adjusted indices based on substring indexingOf the first (for begin) and last (for end) tokens

also with internal we can use the `greedyMode` which will marge consecutive entities of same type regardless of b-boundaries

In [0]:
pipeline_icd10 = Pipeline(
    stages = [
    documentAssembler,
    sentenceDetectorDL,
    tokenizer,
    stopwords,
    word_embeddings,
    clinical_ner,
    ner_converter,
    chunk_embeddings,
    icd10cm_resolution
  ])

model_icd10 = pipeline_icd10.fit(data_ner)


In [0]:
text = light_data['document'][0]

text

Out[76]: 'A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus (T2DM), one prior episode of HTG-induced pancreatitis three years prior to presentation, associated with an acute hepatitis, and obesity with a body mass index (BMI) of 33.5 kg/m2, presented with a one-week history of polyuria, polydipsia, poor appetite, and vomiting. Two weeks prior to presentation, she was treated with a five-day course of amoxicillin for a respiratory tract infection. She was on metformin, glipizide, and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG. She had been on dapagliflozin for six months at the time of presentation. Physical examination on presentation was significant for dry oral mucosa; significantly, her abdominal examination was benign with no tenderness, guarding, or rigidity. Pertinent laboratory findings on admission were: serum glucose 111 mg/dl, bicarbonate 18 mmol/l, anion gap 20, creatinine 0.4 mg/dL, triglycerides 508 mg/dL, total cholesterol 122 mg/dL, glycated hemoglobin (HbA1c) 10%, and venous pH 7.27. Serum lipase was normal at 43 U/L. Serum acetone levels could not be assessed as blood samples kept hemolyzing due to significant lipemia. The patient was initially admitted for starvation ketosis, as she reported poor oral intake for three days prior to admission. However, serum chemistry obtained six hours after presentation revealed her glucose was 186 mg/dL, the anion gap was still elevated at 21, serum bicarbonate was 16 mmol/L, triglyceride level peaked at 2050 mg/dL, and lipase was 52 U/L. The β-hydroxybutyrate level was obtained and found to be elevated at 5.29 mmol/L - the original sample was centrifuged and the chylomicron layer removed prior to analysis due to interference from turbidity caused by lipemia again. The patient was treated with an insulin drip for euDKA and HTG with a reduction in the anion gap to 13 and triglycerides to 1400 mg/dL, within 24 hours. Her euDKA was thought to be precipitated by her respiratory tract infection in the setting of SGLT2 inhibitor use. The patient was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night, 12 units of insulin lispro with meals, and metformin 1000 mg two times a day. It was determined that all SGLT2 inhibitors should be discontinued indefinitely. She had close follow-up with endocrinology post discharge.'

In [0]:
light_pipeline_icd10 = LightPipeline(model_icd10)


In [0]:
import pandas as pd

light_result = light_pipeline_icd10.annotate(text)

df = pd.DataFrame(list(zip(light_result['ner_chunk'], light_result['icd10cm_code'])),
                  columns = ['Problem','ICD10-CM-Code'])

In [0]:
df.head()

Out[63]:

,Problem,ICD10-CM-Code
0,gestational diabetes mellitus,P702
1,type two diabetes mellitus,E1142
2,T2DM,E1121
3,one prior episode of HTG-induced pancreatitis,K860
4,associated with an acute hepatitis,B172


In [0]:
def get_codes (light_model, text, er_code):

  full_light_result = light_model.fullAnnotate(text)

  chunks = []
  codes = []
  begin = []
  end = []
  resolutions=[]

  for chunk, code in zip(full_light_result[0]['ner_chunk'], full_light_result[0][er_code]):
          
      begin.append(chunk.begin)
      end.append(chunk.end)
      chunks.append(chunk.result)
      codes.append(code.result) 
      resolutions.append(code.metadata['all_k_resolutions'])
      
  import pandas as pd

  df = pd.DataFrame({'chunks':chunks, 'begin': begin, 'end':end, 
                    er_code:codes,
                    'resolutions':resolutions})
  
  
  return df


In [0]:
df = get_codes (light_pipeline_icd10, text, 'icd10cm_code')

df

Out[77]:

,chunks,begin,end,icd10cm_code,resolutions
0,gestational diabetes mellitus,39,67,P702,Neonatal diabetes mellitus:::Type 2 diabetes mellitus with hyperglycemia:::Diabetes insipidus
1,type two diabetes mellitus,128,153,E1142,Type 2 diabetes mellitus with diabetic polyneuropathy:::Neonatal diabetes mellitus:::Diabetes in...
2,T2DM,156,159,E1121,Type 2 diabetes mellitus with diabetic nephropathy
3,one prior episode of HTG-induced pancreatitis,163,207,K860,"Alcohol-induced chronic pancreatitis:::Manic episode, unspecified:::Bipolar disorder, current ep..."
4,associated with an acute hepatitis,244,277,B172,"Acute hepatitis E:::Acute viral hepatitis, unspecified:::Chronic hepatitis, unspecified:::Chroni..."
5,obesity with a body mass index,284,313,Z6828,"Body mass index (BMI) 28.0-28.9, adult:::Body mass index (BMI) 36.0-36.9, adult:::Body mass inde..."
6,polyuria,373,380,R358,Other polyuria:::Polydipsia:::Generalized edema:::Bilious vomiting:::Bartter's syndrome
7,polydipsia,383,392,R631,Polydipsia:::Anhedonia:::Galactorrhea
8,poor appetite,395,407,R630,"Anorexia:::Nutritional deficiency, unspecified:::Dysphagia, unspecified:::Nausea:::Decreased libido"
9,vomiting,414,421,R1114,Bilious vomiting:::Vomiting without nausea:::Nausea


In [0]:
df = get_codes (light_pipeline_icd10, text, 'icd10cm_code')

df

Out[66]:

,chunks,begin,end,icd10cm_code,resolutions
0,gestational diabetes mellitus,39,67,P702,Neonatal diabetes mellitus:::Type 2 diabetes mellitus with hyperglycemia:::Diabetes insipidus
1,type two diabetes mellitus,128,153,E1142,Type 2 diabetes mellitus with diabetic polyneuropathy:::Neonatal diabetes mellitus:::Diabetes in...
2,T2DM,156,159,E1121,Type 2 diabetes mellitus with diabetic nephropathy
3,one prior episode of HTG-induced pancreatitis,163,207,K860,"Alcohol-induced chronic pancreatitis:::Manic episode, unspecified:::Bipolar disorder, current ep..."
4,associated with an acute hepatitis,244,277,B172,"Acute hepatitis E:::Acute viral hepatitis, unspecified:::Chronic hepatitis, unspecified:::Chroni..."
5,obesity with a body mass index,284,313,Z6828,"Body mass index (BMI) 28.0-28.9, adult:::Body mass index (BMI) 36.0-36.9, adult:::Body mass inde..."
6,polyuria,373,380,R358,Other polyuria:::Polydipsia:::Generalized edema:::Bilious vomiting:::Bartter's syndrome
7,polydipsia,383,392,R631,Polydipsia:::Anhedonia:::Galactorrhea
8,poor appetite,395,407,R630,"Anorexia:::Nutritional deficiency, unspecified:::Dysphagia, unspecified:::Nausea:::Decreased libido"
9,vomiting,414,421,R1114,Bilious vomiting:::Vomiting without nausea:::Nausea


In [0]:
import pyspark.sql.functions as F

output = model_icd10.transform(data_ner).cache()

output.select(F.explode(F.arrays_zip("ner_chunk.result","ner_chunk.metadata",
                                     "icd10cm_code.result","icd10cm_code.metadata")).alias("icd10cm_result")) \
      .select(F.expr("icd10cm_result['0']").alias("chunk"),
              F.expr("icd10cm_result['1'].entity").alias("entity"),
              F.expr("icd10cm_result['3'].resolved_text").alias("resolved_text"),
              F.expr("icd10cm_result['2']").alias("code"),
              F.expr("icd10cm_result['3'].all_k_resolutions").alias("cms"))\
      .distinct() \
      .toPandas()


Out[67]:

,chunk,entity,resolved_text,code,cms
0,type two diabetes mellitus,PROBLEM,Type 2 diabetes mellitus with diabetic polyneuropathy,E1142,Type 2 diabetes mellitus with diabetic polyneuropathy:::Neonatal diabetes mellitus:::Diabetes in...
1,lipemia,PROBLEM,Glycosuria,R81,Glycosuria:::Pure hyperglyceridemia:::Hyperchylomicronemia
2,precipitated by her respiratory tract infection,PROBLEM,"Respiratory disorder, unspecified",J989,"Respiratory disorder, unspecified:::Acute nasopharyngitis [common cold]:::Diseases of the respir..."
3,initially admitted for starvation ketosis,PROBLEM,Type 2 diabetes mellitus with ketoacidosis with coma,E1111,Type 2 diabetes mellitus with ketoacidosis with coma:::Type 2 diabetes mellitus with hyperosmola...
4,gestational diabetes mellitus,PROBLEM,Neonatal diabetes mellitus,P702,Neonatal diabetes mellitus:::Type 2 diabetes mellitus with hyperglycemia:::Diabetes insipidus
5,vomiting,PROBLEM,Bilious vomiting,R1114,Bilious vomiting:::Vomiting without nausea:::Nausea
6,polydipsia,PROBLEM,Polydipsia,R631,Polydipsia:::Anhedonia:::Galactorrhea
7,elevated,PROBLEM,Elevated Lipoprotein(a),E7841,"Elevated Lipoprotein(a):::Elevated erythrocyte sedimentation rate:::Hyperaldosteronism, unspecif..."
8,significant for dry oral mucosa,PROBLEM,Irritative hyperplasia of oral mucosa,K136,"Irritative hyperplasia of oral mucosa:::Leukoplakia of oral mucosa, including tongue:::Oral subm..."
9,poor appetite,PROBLEM,Anorexia,R630,"Anorexia:::Nutritional deficiency, unspecified:::Dysphagia, unspecified:::Nausea:::Decreased libido"


In [0]:
text = 'He has a starvation ketosis but nothing found for significant for dry oral mucosa'


In [0]:
df = get_codes(light_pipeline_icd10, text, 'icd10cm_code')

df

Out[79]:

,chunks,begin,end,icd10cm_code,resolutions
0,starvation ketosis,9,26,E71121,Propionic acidemia:::Bartter's syndrome:::Hypocalcemia:::Pituitary-dependent Cushing's disease::...
1,significant for dry oral mucosa,50,80,K136,"Irritative hyperplasia of oral mucosa:::Leukoplakia of oral mucosa, including tongue:::Oral subm..."


## Resolved Entities Visualization

In [0]:
sample_text = 'A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus (T2DM), one prior episode of HTG-induced pancreatitis three years prior to presentation, associated with an acute hepatitis, and obesity with a body mass index (BMI) of 33.5 kg/m2, presented with a one-week history of polyuria, polydipsia, poor appetite, and vomiting. Two weeks prior to presentation, she was treated with a five-day course of amoxicillin for a respiratory tract infection.'

full_light_result = light_pipeline_icd10.fullAnnotate(sample_text)


In [0]:
from sparknlp_display import EntityResolverVisualizer

visualizer = EntityResolverVisualizer()

# Change color of an entity label
#visualizer.set_label_colors({'PROBLEM':'#008080'})

vis = visualizer.display(full_light_result[0], 'ner_chunk', 'icd10cm_code', return_html=True)

displayHTML(vis)

A 28-year-old female with a history of gestational diabetes mellitus PROBLEM P702 Neonatal diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus PROBLEM E1142 Type 2 diabetes mellitus with diabetic polyneuropathy ( T2DM PROBLEM E1121 Type 2 diabetes mellitus with diabetic nephropathy ), one prior episode of HTG-induced pancreatitis PROBLEM K860 Alcohol-induced chronic pancreatitis three years prior to presentation, associated with an acute hepatitis PROBLEM B172 Acute hepatitis E , and obesity with a body mass PROBLEM E661 Drug-induced obesity index (BMI) of 33.5 kg/m2, presented with a one-week history of polyuria PROBLEM R358 Other polyuria , polydipsia PROBLEM R631 Polydipsia , poor appetite PROBLEM R630 Anorexia , and vomiting PROBLEM R1114 Bilious vomiting . Two weeks prior to presentation, she was treated with a five-day course of amoxicillin for a respiratory tract infection PROBLEM J00 Acute nasopharyngitis [common cold] .

## CPT Resolver

The Current Procedural Terminology (CPT) code set is a medical code set maintained by the American Medical Association. The CPT code set describes medical, surgical, and diagnostic services and is designed to communicate uniform information about medical services and procedures among physicians, coders, patients, accreditation organizations, and payers for administrative, financial, and analytical purposes.

In [0]:
ner_converter_cpt = NerConverterInternal() \
    .setInputCols(["sentence", "token", "ner"]) \
    .setOutputCol("ner_chunk")\
    .setWhiteList(['TREATMENT','TEST'])\
    .setPreservePosition(False)

cpt_resolution = ChunkEntityResolverModel.pretrained("chunkresolve_cpt_clinical", "en", "clinical/models") \
    .setInputCols(["token", "chunk_embeddings"]) \
    .setOutputCol("cpt_code") \
    .setDistanceFunction("COSINE") \
    .setNeighbours(5)

pipeline_cpt = Pipeline(
    stages = [
    documentAssembler,
    sentenceDetectorDL,
    tokenizer,
    stopwords,
    word_embeddings,
    clinical_ner,
    ner_converter_cpt,
    chunk_embeddings,
    cpt_resolution
  ])

model_cpt = pipeline_cpt.fit(data_ner)


chunkresolve_cpt_clinical download started this may take some time.
Approximate size to download 16.1 MB
[ | ][OK!]

In [0]:
light_model_cpt = LightPipeline(model_cpt)

text = 'The patient needs to have a coronary artery bypass but doctor suggests a abdomen CT at first.'

df = get_codes (light_model_cpt, text, 'cpt_code')

df

Out[82]:

,chunks,begin,end,cpt_code,resolutions
0,coronary artery bypass,28,49,33535,"Coronary artery bypass, using arterial graft(s); 3coronary arterial grafts:::Coronary artery byp..."
1,abdomen CT,73,82,44970,"Laparoscopy, surgical, appendectomy :::Stereotactic biopsy, aspiration, or excision, including b..."


## RxNorm Resolver

In [0]:
posology_ner = MedicalNerModel.pretrained("ner_drugs_large", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner")

ner_converter = NerConverterInternal() \
    .setInputCols(["sentence", "token", "ner"]) \
    .setOutputCol("ner_chunk")\
    .setPreservePosition(False)

rxnorm_resolver = ChunkEntityResolverModel.pretrained('chunkresolve_rxnorm_sbd_clinical', 'en', "clinical/models")\
    .setEnableLevenshtein(True)\
    .setNeighbours(200).setAlternatives(5).setDistanceWeights([3,11,0,0,0,9])\
    .setInputCols('token', 'chunk_embeddings')\
    .setOutputCol('rxnorm_resolution')\
    .setPoolingStrategy("MAX")

pipeline_rxnorm = Pipeline(
    stages = [
    documentAssembler,
    sentenceDetectorDL,
    tokenizer,
    stopwords,
    word_embeddings,
    posology_ner,
    ner_converter,
    chunk_embeddings,
    rxnorm_resolver
  ])


model_rxnorm = pipeline_rxnorm.fit(data_ner)

ner_drugs_large download started this may take some time.
Approximate size to download 13.9 MB
[ | ][OK!]
chunkresolve_rxnorm_sbd_clinical download started this may take some time.
Approximate size to download 17.9 MB
[ | ][OK!]

In [0]:
output = model_rxnorm.transform(data_ner)

result_df = output.select(F.explode(F.arrays_zip("ner_chunk.result","ner_chunk.metadata","rxnorm_resolution.result","rxnorm_resolution.metadata"))\
                          .alias("rxnorm_result")) \
                  .select(F.expr("rxnorm_result['0']").alias("chunk"),
                          F.expr("rxnorm_result['1'].entity").alias("entity"),
                          F.expr("rxnorm_result['3'].all_k_resolutions").alias("target_text"),
                          F.expr("rxnorm_result['2']").alias("code"),
                          F.expr("rxnorm_result['3'].confidence").alias("confidence"))

result_df.show(truncate = 100)

+----------------------------+------+----------------------------------------------------------------------------------------------------+-------+----------+
 chunk|entity| target_text| code|confidence|
+----------------------------+------+----------------------------------------------------------------------------------------------------+-------+----------+
 amoxicillin| DRUG|Amoxicillin 50 MG Oral Tablet [Biomox]:::Amoxicillin 500 MG Oral Capsule [Amix]:::Amoxicillin 250...| 791949| 0.2170|
 metformin| DRUG|Metformin hydrochloride 500 MG Oral Tablet [Glucamet]:::Metformin hydrochloride 850 MG Oral Table...| 105376| 0.2067|
 glipizide| DRUG|Glipizide 5 MG Oral Tablet [Minidiab]:::Glipizide 5 MG Oral Tablet [Glucotrol]:::Glipizide 5 MG O...| 105373| 0.2224|
 dapagliflozin for T2DM| DRUG|dapagliflozin 5 MG / saxagliptin 5 MG Oral Tablet [Qtern]:::dapagliflozin 10 MG / saxagliptin 5 M...|2169276| 0.2532|
atorvastatin and gemfibrozil| DRUG|atorvastatin 10 MG Oral Tablet [Lipitor]:::atorvastatin 20 MG Oral Tablet [Lipitor]:::atorvastati...| 617314| 0.2166|
 dapagliflozin| DRUG|dapagliflozin 5 MG Oral Tablet [Farxiga]:::dapagliflozin 10 MG Oral Tablet [Farxiga]:::dapagliflo...|1486981| 0.3523|
 insulin drip| DRUG|Insulin Lispro 100 UNT/ML Injectable Solution [Humalog]:::Insulin Lispro 100 UNT/ML Cartridge [Hu...| 865098| 0.2250|
 SGLT2 inhibitor| DRUG|C1 esterase inhibitor (human) 500 UNT Injection [Cinryze]:::alpha 1-proteinase inhibitor, human 1...| 809871| 0.2044|
 insulin glargine| DRUG|Insulin Glargine 100 UNT/ML Pen Injector [Lantus]:::Insulin Glargine 300 UNT/ML Pen Injector [Tou...|1359856| 0.2265|
 insulin lispro| DRUG|Insulin Lispro 100 UNT/ML Cartridge [Humalog]:::Insulin Lispro 200 UNT/ML Pen Injector [Humalog]:...|1652648| 0.2340|
 metformin 1000 mg| DRUG|Metformin hydrochloride 500 MG Oral Tablet [Glucamet]:::Metformin hydrochloride 500 MG Oral Table...| 105376| 0.2324|
 SGLT2 inhibitors| DRUG|alpha 1-proteinase inhibitor, human 1 MG Injection [Prolastin]:::C1 esterase inhibitor (human) 50...|1661220| 0.2167|
+----------------------------+------+----------------------------------------------------------------------------------------------------+-------+----------+

In [0]:
df = result_df.toPandas()
df                 

Out[152]:

,chunk,entity,target_text,code,confidence
0,amoxicillin,DRUG,Amoxicillin 50 MG Oral Tablet [Biomox]:::Amoxicillin 500 MG Oral Capsule [Amix]:::Amoxicillin 250 MG Oral Capsule [Amix]:::Amoxicillin 500 MG Oral Capsule [Wymox]:::Amoxicillin 500 MG Oral Capsule [Sumox],791949,0.2170
1,metformin,DRUG,Metformin hydrochloride 500 MG Oral Tablet [Glucamet]:::Metformin hydrochloride 850 MG Oral Tablet [Glucamet]:::Metformin hydrochloride 850 MG Oral Tablet [Glucophage]:::Metformin hydrochloride 625 MG Oral Tablet [Glucophage]:::Metformin hydrochloride 850 MG Oral Tablet [Metforming],105376,0.2067
2,glipizide,DRUG,Glipizide 5 MG Oral Tablet [Minidiab]:::Glipizide 5 MG Oral Tablet [Glucotrol]:::Glipizide 5 MG Oral Tablet [Glibenese]:::Glipizide 10 MG Oral Tablet [Glucotrol]:::Glipizide 2.5 MG Oral Tablet [Minidiab],105373,0.2224
3,dapagliflozin for T2DM,DRUG,dapagliflozin 5 MG / saxagliptin 5 MG Oral Tablet [Qtern]:::dapagliflozin 10 MG / saxagliptin 5 MG Oral Tablet [Qtern]:::dapagliflozin 5 MG / Metformin hydrochloride 500 MG Extended Release Oral Tablet [Xigduo]:::dapagliflozin 5 MG / Metformin hydrochloride 1000 MG Extended Release Oral Tablet [Xigduo]:::dapagliflozin 10 MG / Metformin hydrochloride 500 MG Extended Release Oral Tablet [Xigduo],2169276,0.2532
4,atorvastatin and gemfibrozil,DRUG,atorvastatin 10 MG Oral Tablet [Lipitor]:::atorvastatin 20 MG Oral Tablet [Lipitor]:::atorvastatin 40 MG Oral Tablet [Lipitor]:::atorvastatin 80 MG Oral Tablet [Lipitor]:::Amlodipine 5 MG / atorvastatin 10 MG Oral Tablet [Caduet],617314,0.2166
5,dapagliflozin,DRUG,dapagliflozin 5 MG Oral Tablet [Farxiga]:::dapagliflozin 10 MG Oral Tablet [Farxiga]:::dapagliflozin 5 MG / saxagliptin 5 MG Oral Tablet [Qtern]:::dapagliflozin 10 MG / saxagliptin 5 MG Oral Tablet [Qtern]:::dapagliflozin 5 MG / Metformin hydrochloride 500 MG Extended Release Oral Tablet [Xigduo],1486981,0.3523
6,insulin drip,DRUG,Insulin Lispro 100 UNT/ML Injectable Solution [Humalog]:::Insulin Lispro 100 UNT/ML Cartridge [Humalog]:::3 ML Insulin Lispro 100 UNT/ML Cartridge [Humalog]:::Insulin Lispro 100 UNT/ML Injectable Solution [Admelog]:::insulin degludec 100 UNT/ML Injectable Solution [Tresiba],865098,0.2250
7,SGLT2 inhibitor,DRUG,"C1 esterase inhibitor (human) 500 UNT Injection [Cinryze]:::alpha 1-proteinase inhibitor, human 1 MG Injection [Zemaira]:::alpha 1-proteinase inhibitor, human 1 MG Injection [Aralast]:::alpha 1-proteinase inhibitor, human 1 MG Injection [Glassia]:::C1 esterase inhibitor (human) 500 UNT Injection [Berinert]",809871,0.2044
8,insulin glargine,DRUG,Insulin Glargine 100 UNT/ML Pen Injector [Lantus]:::Insulin Glargine 300 UNT/ML Pen Injector [Toujeo]:::Insulin Glargine 100 UNT/ML Pen Injector [Basaglar]:::3 ML Insulin Glargine 100 UNT/ML Pen Injector [Lantus]:::3 ML Insulin Glargine 300 UNT/ML Pen Injector [Toujeo],1359856,0.2265
9,insulin lispro,DRUG,Insulin Lispro 100 UNT/ML Cartridge [Humalog]:::Insulin Lispro 200 UNT/ML Pen Injector [Humalog]:::Insulin Lispro 100 UNT/ML Pen Injector [Admelog]:::Insulin Lispro 100 UNT/ML Pen Injector [Humalog]:::3 ML Insulin Lispro 100 UNT/ML Cartridge [Humalog],1652648,0.2340


## Snomed Resolver

In [0]:
snomed_ner_converter = NerConverterInternal() \
  .setInputCols(["sentence", "token", "ner"]) \
  .setOutputCol("greedy_chunk")\
  .setWhiteList(['PROBLEM','TEST'])

chunk_embeddings = ChunkEmbeddings()\
  .setInputCols('greedy_chunk', 'embeddings')\
  .setOutputCol('chunk_embeddings')

snomed_resolver = ChunkEntityResolverModel.pretrained("chunkresolve_snomed_findings_clinical","en","clinical/models")\
    .setInputCols("token","chunk_embeddings")\
    .setOutputCol("snomed_resolution")


pipeline_snomed = Pipeline(
    stages = [
    documentAssembler,
    sentenceDetector,
    tokenizer,
    stopwords,
    word_embeddings,
    clinical_ner,
    snomed_ner_converter,
    chunk_embeddings,
    snomed_resolver
  ])

model_snomed = pipeline_snomed.fit(data)


chunkresolve_snomed_findings_clinical download started this may take some time.
Approximate size to download 162.5 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][OK!]

In [0]:
snomed_output = model_snomed.transform(data)

snomed_output.write.mode("overwrite").save("/snomed_temp")

snomed_output = spark.read.load("/snomed_temp")

In [0]:
snomed_output.select(F.explode(F.arrays_zip("greedy_chunk.result","greedy_chunk.metadata","snomed_resolution.result","snomed_resolution.metadata"))\
                     .alias("snomed_result")) \
              .select(F.expr("snomed_result['0']").alias("chunk"),
                      F.expr("snomed_result['1'].entity").alias("entity"),
                      F.expr("snomed_result['3'].all_k_resolutions").alias("target_text"),
                      F.expr("snomed_result['2']").alias("code"),
                      F.expr("snomed_result['3'].confidence").alias("confidence")).show(truncate = 100)

+-----------------------------------------------------------------------------+-------+----------------------------------------------------------------------------------------------------+-----------------+----------+
 chunk| entity| target_text| code|confidence|
+-----------------------------------------------------------------------------+-------+----------------------------------------------------------------------------------------------------+-----------------+----------+
 erythematous skin lesions|PROBLEM|Skin lesion:::Achromic skin lesions of pinta:::Scaly skin:::Skin constricture:::Cratered skin les...| 95324001| 0.0937|
 pruritus|PROBLEM|Pruritus:::Genital pruritus:::Postmenopausal pruritus:::Pruritus hiemalis:::Pruritus ani:::Anogen...| 418363000| 0.1394|
 pruritus|PROBLEM|Pruritus:::Genital pruritus:::Postmenopausal pruritus:::Pruritus hiemalis:::Pruritus ani:::Anogen...| 418363000| 0.1394|
 hypertension|PROBLEM|Hypertension:::Renovascular hypertension:::Idiopathic hypertension:::Venous hypertension:::Resist...| 38341003| 0.1019|
 headache or pain|PROBLEM|Pain:::Headache:::Postchordotomy pain:::Throbbing pain:::Aching headache:::Postspinal headache:::...| 22253000| 0.0953|
 lesion on corner of mouth|PROBLEM|Vesiculoerosive lesion:::Lesion of tongue:::Erythroleukoplakia of mouth:::Lesion of nose:::Lesion...| 708484008| 0.0578|
 nausea and vomiting|PROBLEM|Nausea and vomiting:::Vomiting without nausea:::Nausea:::Intractable nausea and vomiting:::Vomiti...| 16932000| 0.0995|
 perianal irritation|PROBLEM|Perineal irritation:::Vulval irritation:::Skin irritation:::Perianal pain:::Perianal itch:::Vagin...| 281639001| 0.0764|
 insomnia|PROBLEM|Insomnia:::Mood insomnia:::Nonorganic insomnia:::Persistent insomnia:::Psychophysiologic insomnia...| 193462001| 0.1198|
 patient's incisions sternal and right leg|PROBLEM|Laceration of right lower leg:::Varicose veins of right leg:::Stab wound of right lower leg:::Clo...|10963831000119103| 0.0526|
 blood pressure| TEST|Elevated blood pressure:::Abnormal blood pressure:::High blood pressure:::Low blood pressure:::No...| 24184005| 0.0480|
 hematocrit| TEST|Hematocrit:::Stable hematocrit:::Precipitous drop in hematocrit:::Hematocrit - borderline high:::...| 365616005| 0.5722|
 bun and creatinine| TEST|Micropunctum lacrimale:::Creatinine in sample:::Serum creatinine normal:::Serum creatinine low:::...| 95505003| 0.1501|
 prothrombin time level| TEST|Prothrombin time low:::Prothrombin time increased:::Prothrombin time finding:::Prothrombin time n...| 165569003| 0.1356|
 chest x-ray| TEST|Flat chest:::Rigid chest:::Chest hyperinflated:::Chest percussion tympanitic:::Chest mass:::Chest...| 3274008| 0.0560|
small bilateral effusions with mild cardiomegaly and subsegmental atelectasis|PROBLEM|Bilateral pleural effusion (disorder):::Effusion of joint of bilateral ankles:::Effusion of joint...| 425802001| 0.0501|
 bibasilar and electrocardiogram| TEST|Electrocardiogram abnormal:::Electrocardiogram normal:::Electrocardiogram finding:::Electrocardio...| 102594003| 0.0703|
 left atrial enlargement|PROBLEM|Left atrial enlargement:::Right atrial enlargement:::Left atrial hypertrophy:::Left atrial dilata...| 67741000119109| 0.1804|
 acute ischemic changes on electrocardiogram|PROBLEM|Postoperative electrocardiogram changes:::Acute ischemic renal failure:::Potential for acute isch...| 251144001| 0.1238|
 hypertension|PROBLEM|Hypertension:::Renovascular hypertension:::Idiopathic hypertension:::Venous hypertension:::Resist...| 38341003| 0.1019|
+-----------------------------------------------------------------------------+-------+----------------------------------------------------------------------------------------------------+-----------------+----------+
only showing top 20 rows

## ICD10 + RxNorm with Multiple NERs

In [0]:
notes = [
'Pentamidine 300 mg IV q . 36 hours , Pentamidine nasal wash 60 mg per 6 ml of sterile water q.d . , voriconazole 200 mg p.o . b.i.d . , acyclovir 400 mg p.o . b.i.d . , cyclosporine 50 mg p.o . b.i.d . , prednisone 60 mg p.o . q.d . , GCSF 480 mcg IV q.d . , Epogen 40,000 units subcu q . week , Protonix 40 mg q.d . , Simethicone 80 mg p.o . q . 8 , nitroglycerin paste 1 " ; q . 4 h . p.r.n . , flunisolide nasal inhaler , 2 puffs q . 8 , OxyCodone 10-15 mg p.o . q . 6 p.r.n . , Sudafed 30 mg q . 6 p.o . p.r.n . , Fluconazole 2% cream b.i.d . to erythematous skin lesions , Ditropan 5 mg p.o . b.i.d . , Tylenol 650 mg p.o . q . 4 h . p.r.n . , Ambien 5-10 mg p.o . q . h.s . p.r.n . , Neurontin 100 mg q . a.m . , 200 mg q . p.m . , Aquaphor cream b.i.d . p.r.n . , Lotrimin 1% cream b.i.d . to feet , Dulcolax 5-10 mg p.o . q.d . p.r.n . , Phoslo 667 mg p.o . t.i.d . , Peridex 0.12% , 15 ml p.o . b.i.d . mouthwash , Benadryl 25-50 mg q . 4-6 h . p.r.n . pruritus , Sarna cream q.d . p.r.n . pruritus , Nystatin 5 ml p.o . q.i.d . swish and !',
'Albuterol nebulizers 2.5 mg q.4h . and Atrovent nebulizers 0.5 mg q.4h . , please alternate albuterol and Atrovent ; Rocaltrol 0.25 mcg per NG tube q.d .; calcium carbonate 1250 mg per NG tube q.i.d .; vitamin B12 1000 mcg IM q . month , next dose is due Nov 18 ; diltiazem 60 mg per NG tube t.i.d .; ferrous sulfate 300 mg per NG t.i.d .; Haldol 5 mg IV q.h.s .; hydralazine 10 mg IV q.6h . p.r.n . hypertension ; lisinopril 10 mg per NG tube q.d .; Ativan 1 mg per NG tube q.h.s .; Lopressor 25 mg per NG tube t.i.d .; Zantac 150 mg per NG tube b.i.d .; multivitamin 10 ml per NG tube q.d .; Macrodantin 100 mg per NG tube q.i.d . x 10 days beginning on 11/3/00 .',
'Tylenol 650 mg p.o . q . 4-6h p.r.n . headache or pain ; acyclovir 400 mg p.o . t.i.d .; acyclovir topical t.i.d . to be applied to lesion on corner of mouth ; Peridex 15 ml p.o . b.i.d .; Mycelex 1 troche p.o . t.i.d .; g-csf 404 mcg subcu q.d .; folic acid 1 mg p.o . q.d .; lorazepam 1-2 mg p.o . q . 4-6h p.r.n . nausea and vomiting ; Miracle Cream topical q.d . p.r.n . perianal irritation ; Eucerin Cream topical b.i.d .; Zantac 150 mg p.o . b.i.d .; Restoril 15-30 mg p.o . q . h.s . p.r.n . insomnia ; multivitamin 1 tablet p.o . q.d .; viscous lidocaine 15 ml p.o . q . 3h can be applied to corner of mouth or lips p.r.n . pain control .',
'The patient\'s incisions sternal and right leg were clean and healing well , normal sinus rhythm at 70-80 , with blood pressure 98-110/60 and patient was doing well , recovering , ambulating , tolerating regular diet and last hematocrit prior to discharge was 39% with a BUN and creatinine of 15 and 1.0 , prothrombin time level of 13.8 , chest X-ray prior to discharge showed small bilateral effusions with mild cardiomegaly and subsegmental atelectasis bibasilar and electrocardiogram showed normal sinus rhythm with left atrial enlargement and no acute ischemic changes on electrocardiogram .',
'This is an 82 - year-old male with a history of prior tobacco use , hypertension , chronic renal insufficiency , COPD , gastritis , and TIA who initially presented to Braintree with a non-ST elevation MI and Guaiac positive stools , transferred to St . Margaret\'s Center for Women & Infants for cardiac catheterization with PTCA to mid LAD lesion complicated by hypotension and bradycardia requiring Atropine , IV fluids and transient dopamine possibly secondary to vagal reaction , subsequently transferred to CCU for close monitoring , hemodynamically stable at the time of admission to the CCU .',
'O2 95% on 3L NC mixed Quinn 82% genrl : in nad , resting comfortably heent : perrla ( 4->3 mm ) bilaterally , blind in right visual field , eomi , dry mm , ? thrush neck : no bruits cv : rrr , no m/r/g , faint s1/s2 pulm : cta bilaterally abd : midline scar ( from urostomy ) , nabs , soft , appears distended but patient denies , ostomy RLQ c/d/i , NT to palpation back : right flank urostomy tube , c/d/i , nt to palpation extr : no Gardner neuro : a , ox3 , wiggles toes bilaterally , unable to lift LE , 06-12 grip bilaterally w/ UE , decrease sensation to soft touch in left',
'Is notable for an inferior myocardial infarction , restrictive and obstructive lung disease with an FEV1 of . 9 and FVC of 1.34 and a moderate at best response to bronchodilators , and a negative sestamibi scan in May , 1999 apart from a severe fixed inferolateral defect , systolic dysfunction with recent echocardiography revealing an LVID of 62 mm . and ejection fraction of 28 percent , moderate mitral regurgitation and mild-to-moderate aortic stenosis with a peak gradient of 33 and a mean gradient of 19 and a valve area of 1.4 cm . squared .',
'This is a 47 - year-old male with a past medical history of type 2 diabetes , high cholesterol , hypertension , and coronary artery disease , status post percutaneous transluminal coronary angioplasty times two , who presented with acute coronary syndrome refractory to medical treatment and TNK , now status post Angio-Jet percutaneous transluminal coronary angioplasty and stent of proximal left anterior descending artery and percutaneous transluminal coronary angioplasty of first diagonal with intra-aortic balloon pump placement .',
'Clinical progression of skin and sinus infection on maximal antimicrobial therapy continued , with emergence on November 20 of a new right-sided ptosis in association with a left homonymous hemianopsia , and fleeting confusion while febrile , prompting head MRI which revealed a large 5 x 2 x 4.3 cm region in the right occipital lobe of hemorrhage and edema , with dural and , likely , leptomeningeal enhancement in association with small foci in the right cerebellum and pons , concerning for early lesions of similar type .',
'The patient had an echocardiogram on day two of admission , which revealed a mildly dilated left atrium , mild symmetric LVH , normal LV cavity size , mild region LV systolic dysfunction , arresting regional wall motion abnormality including focal apical hypokinesis , a normal right ventricular chamber size and free wall motion , a moderately dilated aortic root , a mildly dilated ascending aorta , normal aortic valve leaflet , normal mitral valve leaflet and no pericardial effusions .',
'The patient is a 65-year-old man with refractory CLL , status post non-myeloblative stem cell transplant approximately nine months prior to admission , and status post prolonged recent Retelk County Medical Center stay for Acanthamoeba infection of skin and sinuses , complicated by ARS due to medication toxicity , as well as GVHD and recent CMV infection , readmitted for new fever , increasing creatinine , hepatomegaly and fluid surge spacing , in the setting of hyponatremia .',
'Tylenol 650 mg p.o . q.4h . p.r.n . , Benadryl 25 mg p.o . q.h.s . p.r.n . , Colace 100 mg p.o . q.i.d . , Nortriptyline 25 mg p.o . q.h.s . , Simvastatin 10 mg p.o . q.h.s . , Metamucil one packet p.o . b.i.d . p.r.n . , Neurontin 300 mg p.o . t.i.d . , Levsinex 0.375 mg p.o . q.12h . , Lisinopril / hydrochlorothiazide 20/25 mg p.o . q.d . , hydrocortisone topical ointment to affected areas , MS Contin 30 mg p.o . b.i.d . , MSIR 15 to 30 mg p.o . q.4h . p.r.n . pain .',
'Aspirin 325 q.d . ; albuterol nebs 2.5 mg q . 4h ; Colace 100 mg b.i.d . ; heparin 5,000 units subcu b.i.d . ; Synthroid 200 mcg q.d . ; Ocean Spray 2 sprays q . i.d . ; simvastatin 10 mg q . h.s . ; Flovent 220 mcg 2 puffs b.i.d . ; Zantac 150 b.i.d . ; nystatin ointment to the gluteal fold b.i.d . ; Lisinopril 20 mg q.d . ; Mestinon controlled release 180 q . h.s . ; Mestinon 30 mg q . 4h while awake ; prednisone 60 mg p.o . q . IM ; Atrovent nebs 0.5 mg q . i.d .',
'An echocardiogram was obtained on 4-26 which showed concentric left ventricular hypertrophy with normal _____ left ventricular function , severe right ventricular dilatation with septal hypokinesis and flattening with a question of right ventricular apical clot raised with mild aortic stenosis , severe tricuspid regurgitation and increased pulmonary artery pressure of approximately 70 millimeters , consistent with fairly severe pulmonary hypertension .',
'1 ) CV ( R ) finished amio IV load then started on po , agressive lytes ; although interrogation showed >100 episodes of VT ( as / x ) , pt prefers med therapy as opposed to ablation ( I ) enzymes mildly elevated but not actively ischemic ; lipids , ASA , statin , BB ; Adenosine thal 1/4 and echo 1/4 to look for signs of ischemia as active cause for VT ( P ) JVP at angle of jaw 1/4 -- > giving 20 Lasix ; dig level 1/4 1.3 -- > 1/2 dose as on Amio',
'sodium 141 , potassium 3.5 , chloride 107 , bicarbonates 23.8 , BUN 23 , creatinine 1.1 , glucose 165 , PO2 377 , PCO2 32 , PH 7.50 , asomus 298 , toxic screen negative , white blood cell count 11.1 , hematocrit 39.6 , platelet count 137 , prothrombin time 25.2 , INR 4.3 , partial thromboplastin time 34.7 , urinalysis 1+ albumin , 0-5 high link caths , cervical spine negative , pelvis negative , lumbar spine ; negative , thoracic spine negative .',
]

In [0]:
data = spark.createDataFrame([(i,n.lower()) for i,n in enumerate(notes)]).toDF('doc_id', 'text')

data.show(truncate=50)

+------+--------------------------------------------------+
doc_id| text|
+------+--------------------------------------------------+
 0|pentamidine 300 mg iv q . 36 hours , pentamidin...|
 1|albuterol nebulizers 2.5 mg q.4h . and atrovent...|
 2|tylenol 650 mg p.o . q . 4-6h p.r.n . headache ...|
 3|the patient's incisions sternal and right leg w...|
 4|this is an 82 - year-old male with a history of...|
 5|o2 95% on 3l nc mixed quinn 82% genrl : in nad ...|
 6|is notable for an inferior myocardial infarctio...|
 7|this is a 47 - year-old male with a past medica...|
 8|clinical progression of skin and sinus infectio...|
 9|the patient had an echocardiogram on day two of...|
 10|the patient is a 65-year-old man with refractor...|
 11|tylenol 650 mg p.o . q.4h . p.r.n . , benadryl ...|
 12|aspirin 325 q.d . ; albuterol nebs 2.5 mg q . 4...|
 13|an echocardiogram was obtained on 4-26 which sh...|
 14|1 ) cv ( r ) finished amio iv load then started...|
 15|sodium 141 , potassium 3.5 , chloride 107 , bic...|
+------+--------------------------------------------------+

In [0]:
from IPython.core.display import display, HTML

html_output=""
for i, d in enumerate(notes):
    html_output += f'Note {i}:'
    html_output +='<div style="border:2px solid #747474; background-color: #e3e3e3; margin: 5px; padding: 10px">'
    html_output += d
    html_output += '</div><br/>'

displayHTML(html_output)

Note 0: Pentamidine 300 mg IV q . 36 hours , Pentamidine nasal wash 60 mg per 6 ml of sterile water q.d . , voriconazole 200 mg p.o . b.i.d . , acyclovir 400 mg p.o . b.i.d . , cyclosporine 50 mg p.o . b.i.d . , prednisone 60 mg p.o . q.d . , GCSF 480 mcg IV q.d . , Epogen 40,000 units subcu q . week , Protonix 40 mg q.d . , Simethicone 80 mg p.o . q . 8 , nitroglycerin paste 1 " ; q . 4 h . p.r.n . , flunisolide nasal inhaler , 2 puffs q . 8 , OxyCodone 10-15 mg p.o . q . 6 p.r.n . , Sudafed 30 mg q . 6 p.o . p.r.n . , Fluconazole 2% cream b.i.d . to erythematous skin lesions , Ditropan 5 mg p.o . b.i.d . , Tylenol 650 mg p.o . q . 4 h . p.r.n . , Ambien 5-10 mg p.o . q . h.s . p.r.n . , Neurontin 100 mg q . a.m . , 200 mg q . p.m . , Aquaphor cream b.i.d . p.r.n . , Lotrimin 1% cream b.i.d . to feet , Dulcolax 5-10 mg p.o . q.d . p.r.n . , Phoslo 667 mg p.o . t.i.d . , Peridex 0.12% , 15 ml p.o . b.i.d . mouthwash , Benadryl 25-50 mg q . 4-6 h . p.r.n . pruritus , Sarna cream q.d . p.r.n . pruritus , Nystatin 5 ml p.o . q.i.d . swish and ! Note 1: Albuterol nebulizers 2.5 mg q.4h . and Atrovent nebulizers 0.5 mg q.4h . , please alternate albuterol and Atrovent ; Rocaltrol 0.25 mcg per NG tube q.d .; calcium carbonate 1250 mg per NG tube q.i.d .; vitamin B12 1000 mcg IM q . month , next dose is due Nov 18 ; diltiazem 60 mg per NG tube t.i.d .; ferrous sulfate 300 mg per NG t.i.d .; Haldol 5 mg IV q.h.s .; hydralazine 10 mg IV q.6h . p.r.n . hypertension ; lisinopril 10 mg per NG tube q.d .; Ativan 1 mg per NG tube q.h.s .; Lopressor 25 mg per NG tube t.i.d .; Zantac 150 mg per NG tube b.i.d .; multivitamin 10 ml per NG tube q.d .; Macrodantin 100 mg per NG tube q.i.d . x 10 days beginning on 11/3/00 . Note 2: Tylenol 650 mg p.o . q . 4-6h p.r.n . headache or pain ; acyclovir 400 mg p.o . t.i.d .; acyclovir topical t.i.d . to be applied to lesion on corner of mouth ; Peridex 15 ml p.o . b.i.d .; Mycelex 1 troche p.o . t.i.d .; g-csf 404 mcg subcu q.d .; folic acid 1 mg p.o . q.d .; lorazepam 1-2 mg p.o . q . 4-6h p.r.n . nausea and vomiting ; Miracle Cream topical q.d . p.r.n . perianal irritation ; Eucerin Cream topical b.i.d .; Zantac 150 mg p.o . b.i.d .; Restoril 15-30 mg p.o . q . h.s . p.r.n . insomnia ; multivitamin 1 tablet p.o . q.d .; viscous lidocaine 15 ml p.o . q . 3h can be applied to corner of mouth or lips p.r.n . pain control . Note 3: The patient's incisions sternal and right leg were clean and healing well , normal sinus rhythm at 70-80 , with blood pressure 98-110/60 and patient was doing well , recovering , ambulating , tolerating regular diet and last hematocrit prior to discharge was 39% with a BUN and creatinine of 15 and 1.0 , prothrombin time level of 13.8 , chest X-ray prior to discharge showed small bilateral effusions with mild cardiomegaly and subsegmental atelectasis bibasilar and electrocardiogram showed normal sinus rhythm with left atrial enlargement and no acute ischemic changes on electrocardiogram . Note 4: This is an 82 - year-old male with a history of prior tobacco use , hypertension , chronic renal insufficiency , COPD , gastritis , and TIA who initially presented to Braintree with a non-ST elevation MI and Guaiac positive stools , transferred to St . Margaret's Center for Women & Infants for cardiac catheterization with PTCA to mid LAD lesion complicated by hypotension and bradycardia requiring Atropine , IV fluids and transient dopamine possibly secondary to vagal reaction , subsequently transferred to CCU for close monitoring , hemodynamically stable at the time of admission to the CCU . Note 5: O2 95% on 3L NC mixed Quinn 82% genrl : in nad , resting comfortably heent : perrla ( 4->3 mm ) bilaterally , blind in right visual field , eomi , dry mm , ? thrush neck : no bruits cv : rrr , no m/r/g , faint s1/s2 pulm : cta bilaterally abd : midline scar ( from urostomy ) , nabs , soft , appears distended but patient denies , ostomy RLQ c/d/i , NT to palpation back : right flank urostomy t

let's build a SparkNLP pipeline with the following stages:

`DocumentAssembler`: Entry annotator for our pipelines; it creates the data structure for the Annotation Framework

`SentenceDetector`: Annotator to pragmatically separate complete sentences inside each document

`Tokenizer`: Annotator to separate sentences in tokens (generally words)

`StopWordsCleaner`: Annotator to remove words defined as StopWords in SparkML

`WordEmbeddings`: Vectorization of word tokens, in this case using word embeddings trained from PubMed, ICD10 and other clinical resources.

`ChunkEmbeddings`: Aggregates the WordEmbeddings for each NER Chunk

`JSL NER + NerConverter`: This annotators return Chunks related to jsl_ner (generic ner) 

`Drug NER + NerConverter`: This annotators return Chunks related to drugs

`ChunkEntityResolver`: Annotator that performs search for the KNNs, in this case trained from ICDO Histology Behavior.

In [0]:
# Annotators responsible for the Cancer Genetics Entity Recognition task

jslNer = MedicalNerModel.pretrained('ner_jsl', 'en', "clinical/models")\
    .setInputCols('sentence', 'token', 'embeddings')\
    .setOutputCol('ner_jsl')

drugNer = MedicalNerModel.pretrained('ner_drugs', 'en', "clinical/models")\
    .setInputCols('sentence', 'token', 'embeddings')\
    .setOutputCol('ner_drug')

ner_jsl download started this may take some time.
Approximate size to download 14 MB
[ | ][ / ][OK!]
ner_drugs download started this may take some time.
Approximate size to download 13.8 MB
[ | ][ / ][ — ][ \ ][ | ][OK!]

In [0]:

#Converter annotators transform IOB tags into full chunks (sequence set of tokens) tagged with `entity` metadata

jslConverter = NerConverter()\
    .setInputCols('sentence', 'token', 'ner_jsl')\
    .setOutputCol('chunk_jsl')\
    .setWhiteList(["Diagnosis"])

drugConverter = NerConverter()\
    .setInputCols('sentence', 'token', 'ner_drug')\
    .setOutputCol('chunk_drug')

In [0]:

#ChunkEmbeddings annotators aggregate embeddings for each token in the chunk

jslChunkEmbeddings = ChunkEmbeddings()\
  .setInputCols('chunk_jsl', 'embeddings')\
  .setOutputCol('chunk_embs_jsl')

drugChunkEmbeddings = ChunkEmbeddings()\
  .setInputCols('chunk_drug', 'embeddings')\
  .setOutputCol('chunk_embs_drug')

In [0]:
# Entity Resolution Pretrained Models

icd10cmResolver2 = ChunkEntityResolverModel.pretrained('chunkresolve_icd10cm_diseases_clinical', 'en', "clinical/models")\
    .setEnableLevenshtein(True)\
    .setNeighbours(200).setAlternatives(5).setDistanceWeights([3,3,2,0,0,7])\
    .setInputCols('token', 'chunk_embs_jsl')\
    .setOutputCol('icd10cm_resolution')

rxnormResolver2 = ChunkEntityResolverModel()\
    .pretrained('chunkresolve_rxnorm_scd_clinical', 'en', "clinical/models")\
    .setEnableLevenshtein(True)\
    .setNeighbours(200).setAlternatives(5).setDistanceWeights([3,3,2,0,0,7])\
    .setInputCols('token', 'chunk_embs_drug')\
    .setOutputCol('rxnorm_resolution')\

chunkresolve_icd10cm_diseases_clinical download started this may take some time.
Approximate size to download 100.6 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][OK!]
chunkresolve_rxnorm_scd_clinical download started this may take some time.
Approximate size to download 28.5 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][OK!]

In [0]:
# Tokenizer splits words in a relevant format for NLP

tokenizer = Tokenizer()\
  .setInputCols(["sentence"])\
  .setOutputCol("raw_token")\

# Tokenizer splits words in a relevant format for NLP

stopwords = StopWordsCleaner()\
  .setInputCols(["raw_token"])\
  .setOutputCol("token")


pipelineFull = Pipeline().setStages([
    documentAssembler, 
    sentenceDetector, 
    tokenizer, 
    stopwords, 
    word_embeddings, 
    jslNer,
    drugNer,
    jslConverter,
    drugConverter,
    jslChunkEmbeddings, 
    drugChunkEmbeddings,
    icd10cmResolver2,
    rxnormResolver2
])

In [0]:
# Persisiting temporarily to keep DAG size and resource usage low (Word Embeddings are Resource Intensive)
pipelineModelFull = pipelineFull.fit(data)

output = pipelineModelFull.transform(data)


In [0]:
output.write.mode("overwrite").save("/temp")

output = spark.read.load("/temp")

In [0]:
output.show()

+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
doc_id| text| document| sentence| raw_token| token| embeddings| ner_jsl| ner_drug| chunk_jsl| chunk_drug| chunk_embs_jsl| chunk_embs_drug| icd10cm_resolution| rxnorm_resolution|
+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 0|pentamidine 300 m...|[[document, 0, 10...|[[document, 0, 24...|[[token, 0, 10, p...|[[token, 0, 10, p...|[[word_embeddings...|[[named_entity, 0...|[[named_entity, 0...| []|[[chunk, 0, 10, p...| []|[[word_embeddings...| []|[[entity, 0, 10, ...|
 1|albuterol nebuliz...|[[document, 0, 66...|[[document, 0, 33...|[[token, 0, 8, al...|[[token, 0, 8, al...|[[word_embeddings...|[[named_entity, 0...|[[named_entity, 0...|[[chunk, 400, 411...|[[chunk, 0, 8, al...|[[word_embeddings...|[[word_embeddings...|[[entity, 400, 41...|[[entity, 0, 8, 1...|
 2|tylenol 650 mg p....|[[document, 0, 64...|[[document, 0, 19...|[[token, 0, 6, ty...|[[token, 0, 6, ty...|[[word_embeddings...|[[named_entity, 0...|[[named_entity, 0...|[[chunk, 499, 506...|[[chunk, 0, 6, ty...|[[word_embeddings...|[[word_embeddings...|[[entity, 499, 50...|[[entity, 0, 6, 3...|
 3|the patient's inc...|[[document, 0, 59...|[[document, 0, 59...|[[token, 0, 2, th...|[[token, 4, 12, p...|[[word_embeddings...|[[named_entity, 4...|[[named_entity, 4...| []|[[chunk, 278, 287...| []|[[word_embeddings...| []|[[entity, 278, 28...|
 12|aspirin 325 q.d ....|[[document, 0, 46...|[[document, 0, 16...|[[token, 0, 6, as...|[[token, 0, 6, as...|[[word_embeddings...|[[named_entity, 0...|[[named_entity, 0...| []|[[chunk, 0, 6, as...| []|[[word_embeddings...| []|[[entity, 0, 6, 1...|
 13|an echocardiogram...|[[document, 0, 45...|[[document, 0, 45...|[[token, 0, 1, an...|[[token, 3, 16, e...|[[word_embeddings...|[[named_entity, 3...|[[named_entity, 3...|[[chunk, 432, 453...| []|[[word_embeddings...| []|[[entity, 432, 45...| []|
 14|1 ) cv ( r ) fini...|[[document, 0, 44...|[[document, 0, 72...|[[token, 0, 0, 1,...|[[token, 0, 0, 1,...|[[word_embeddings...|[[named_entity, 0...|[[named_entity, 0...|[[chunk, 230, 237...|[[chunk, 270, 278...|[[word_embeddings...|[[word_embeddings...|[[entity, 230, 23...|[[entity, 270, 27...|
 15|sodium 141 , pota...|[[document, 0, 44...|[[document, 0, 41...|[[token, 0, 5, so...|[[token, 0, 5, so...|[[word_embeddings...|[[named_entity, 0...|[[named_entity, 0...| []|[[chunk, 0, 5, so...| []|[[word_embeddings...| []|[[entity, 0, 5, 3...|
 4|this is an 82 - y...|[[document, 0, 59...|[[document, 0, 25...|[[token, 0, 3, th...|[[token, 11, 12, ...|[[word_embeddings...|[[named_entity, 1...|[[named_entity, 1...|[[chunk, 68, 79, ...|[[chunk, 400, 407...|[[word_embeddings...|[[word_embeddings...|[[entity, 68, 79,...|[[entity, 400, 40...|
 5|o2 95% on 3l nc m...|[[document, 0, 57...|[[document, 0, 57...|[[token, 0, 1, o2...|[[token, 0, 1, o2...|[[word_embeddings...|[[named_entity, 0...|[[named_entity, 0...| []| []| []| []| []| []|
 6|is notable for an...|[[document, 0, 54...|[[document, 0, 10...|[[token, 0, 1, is...|[[token, 3, 9, no...|[[word_embeddings...|[[named_entity, 3...|[[named_entity, 3...|[[chunk, 67, 103,...| []|[[word_embeddings...| []|[[entity, 67, 103...| []|
 7|this is a 47 - ye...|[[document, 0, 53...|[[document, 0, 53...|[[token, 0, 3, th...|[[token, 10, 11, ...|[[word_embeddings...|[[named_entity, 1...|[[named_entity, 1...| []|[[chunk, 83, 93, ...| []|[[word_embeddings...| []|[[entity, 83, 93,...|
 8|clinical progress...|[[document, 0, 52...|[[document, 0, 52...|[[token, 0, 7, cl...|[[token,

In [0]:
# lets see what would have happened if we hadn't persisted the pipeline at disk. 
output = pipelineModelFull.transform(data)

In [0]:
output.show()
## 1.0 vs 18.39 seconds for the first 20 rows (about x20 faster)

+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
doc_id| text| document| sentence| raw_token| token| embeddings| ner_jsl| ner_drug| chunk_jsl| chunk_drug| chunk_embs_jsl| chunk_embs_drug| icd10cm_resolution| rxnorm_resolution|
+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 0|pentamidine 300 m...|[[document, 0, 10...|[[document, 0, 24...|[[token, 0, 10, p...|[[token, 0, 10, p...|[[word_embeddings...|[[named_entity, 0...|[[named_entity, 0...| []|[[chunk, 0, 10, p...| []|[[word_embeddings...| []|[[entity, 0, 10, ...|
 1|albuterol nebuliz...|[[document, 0, 66...|[[document, 0, 33...|[[token, 0, 8, al...|[[token, 0, 8, al...|[[word_embeddings...|[[named_entity, 0...|[[named_entity, 0...|[[chunk, 400, 411...|[[chunk, 0, 8, al...|[[word_embeddings...|[[word_embeddings...|[[entity, 400, 41...|[[entity, 0, 8, 1...|
 2|tylenol 650 mg p....|[[document, 0, 64...|[[document, 0, 19...|[[token, 0, 6, ty...|[[token, 0, 6, ty...|[[word_embeddings...|[[named_entity, 0...|[[named_entity, 0...|[[chunk, 499, 506...|[[chunk, 0, 6, ty...|[[word_embeddings...|[[word_embeddings...|[[entity, 499, 50...|[[entity, 0, 6, 3...|
 3|the patient's inc...|[[document, 0, 59...|[[document, 0, 59...|[[token, 0, 2, th...|[[token, 4, 12, p...|[[word_embeddings...|[[named_entity, 4...|[[named_entity, 4...| []|[[chunk, 278, 287...| []|[[word_embeddings...| []|[[entity, 278, 28...|
 4|this is an 82 - y...|[[document, 0, 59...|[[document, 0, 25...|[[token, 0, 3, th...|[[token, 11, 12, ...|[[word_embeddings...|[[named_entity, 1...|[[named_entity, 1...|[[chunk, 68, 79, ...|[[chunk, 400, 407...|[[word_embeddings...|[[word_embeddings...|[[entity, 68, 79,...|[[entity, 400, 40...|
 5|o2 95% on 3l nc m...|[[document, 0, 57...|[[document, 0, 57...|[[token, 0, 1, o2...|[[token, 0, 1, o2...|[[word_embeddings...|[[named_entity, 0...|[[named_entity, 0...| []| []| []| []| []| []|
 6|is notable for an...|[[document, 0, 54...|[[document, 0, 10...|[[token, 0, 1, is...|[[token, 3, 9, no...|[[word_embeddings...|[[named_entity, 3...|[[named_entity, 3...|[[chunk, 67, 103,...| []|[[word_embeddings...| []|[[entity, 67, 103...| []|
 7|this is a 47 - ye...|[[document, 0, 53...|[[document, 0, 53...|[[token, 0, 3, th...|[[token, 10, 11, ...|[[word_embeddings...|[[named_entity, 1...|[[named_entity, 1...| []|[[chunk, 83, 93, ...| []|[[word_embeddings...| []|[[entity, 83, 93,...|
 8|clinical progress...|[[document, 0, 52...|[[document, 0, 52...|[[token, 0, 7, cl...|[[token, 0, 7, cl...|[[word_embeddings...|[[named_entity, 0...|[[named_entity, 0...| []| []| []| []| []| []|
 9|the patient had a...|[[document, 0, 48...|[[document, 0, 48...|[[token, 0, 2, th...|[[token, 4, 10, p...|[[word_embeddings...|[[named_entity, 4...|[[named_entity, 4...|[[chunk, 439, 487...| []|[[word_embeddings...| []|[[entity, 439, 48...| []|
 10|the patient is a ...|[[document, 0, 48...|[[document, 0, 48...|[[token, 0, 2, th...|[[token, 4, 10, p...|[[word_embeddings...|[[named_entity, 4...|[[named_entity, 4...|[[chunk, 223, 264...|[[chunk, 397, 406...|[[word_embeddings...|[[word_embeddings...|[[entity, 223, 26...|[[entity, 397, 40...|
 11|tylenol 650 mg p....|[[document, 0, 47...|[[document, 0, 19...|[[token, 0, 6, ty...|[[token, 0, 6, ty...|[[word_embeddings...|[[named_entity, 0...|[[named_entity, 0...| []|[[chunk, 0, 6, ty...| []|[[word_embeddings...| []|[[entity, 0, 6, 3...|
 12|aspirin 325 q.d ....|[[document, 0, 46...|[[document, 0, 16...|[[token, 0, 6, as...|[[token, 0, 6, as...|[[word_embeddings...|[[named_entity, 0.

In [0]:
def quick_metadata_analysis(df, doc_field, chunk_field, code_fields):
    code_res_meta = ", ".join([f"{cf}.metadata" for cf in code_fields])
    expression = f"explode(arrays_zip({chunk_field}.begin, {chunk_field}.end, {chunk_field}.result, {chunk_field}.metadata, "+code_res_meta+")) as a"
    top_n_rest = [(f"float(a['{i+4}'].confidence) as {(cf.split('_')[0])}_conf",
                    f"arrays_zip(split(a['{i+4}'].all_k_results,':::'),split(a['{i+4}'].all_k_resolutions,':::')) as {cf.split('_')[0]+'_opts'}")
                    for i, cf in enumerate(code_fields)]
    top_n_rest_args = []
    for tr in top_n_rest:
        for t in tr:
            top_n_rest_args.append(t)
    return df.selectExpr(doc_field, expression) \
        .orderBy('doc_id', F.expr("a['0']"), F.expr("a['1']"))\
        .selectExpr(f"concat_ws('::',{doc_field},a['0'],a['1']) as coords", "a['2'] as chunk","a['3'].entity as entity", *top_n_rest_args)

In [0]:
icd10cm_analysis = quick_metadata_analysis(output, 'doc_id', 'chunk_jsl',['icd10cm_resolution']).toPandas()

/databricks/spark/python/pyspark/sql/pandas/conversion.py:93: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
 Unable to convert the field icd10cm_opts. If this column is not necessary, you may consider dropping it or converting to primitive type before the conversion.
Context: Unsupported type in conversion to Arrow: ArrayType(StructType(List(StructField(0,StringType,true),StructField(1,StringType,true))),false)
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
 warnings.warn(msg)

In [0]:
rxnorm_analysis = quick_metadata_analysis(output, 'doc_id', 'chunk_drug',['rxnorm_resolution']).toPandas()

/databricks/spark/python/pyspark/sql/pandas/conversion.py:93: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
 Unable to convert the field rxnorm_opts. If this column is not necessary, you may consider dropping it or converting to primitive type before the conversion.
Context: Unsupported type in conversion to Arrow: ArrayType(StructType(List(StructField(0,StringType,true),StructField(1,StringType,true))),false)
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
 warnings.warn(msg)

In [0]:
pd.set_option('display.max_colwidth', 250)
pd.set_option('display.max_rows', 500)

In [0]:
icd10cm_analysis[icd10cm_analysis.icd10cm_conf>0.4]

Out[183]:

,coords,chunk,entity,icd10cm_conf,icd10cm_opts
1,2::499::506,insomnia,Diagnosis,0.9050,"[(G4700, Insomnia, unspecified), (G4709, Other insomnia), (F5102, Adjustment insomnia), (F5101, Primary insomnia), (F5109, Other insomnia not due to a substance or known physiological condition)]"
4,4::120::128,gastritis,Diagnosis,0.4680,"[(K2970, Gastritis, unspecified, without bleeding), (B9681, Helicobacter pylori [H. pylori] as the cause of diseases classified elsewhere), (K2900, Acute gastritis without bleeding), (A084, Viral intestinal infection, unspecified), (K2960, Other ..."
5,6::67::103,obstructive lung disease with an fev1,Diagnosis,0.4564,"[(J670, Farmer's lung), (J984, Other disorders of lung), (J449, Chronic obstructive pulmonary disease, unspecified), (J849, Interstitial pulmonary disease, unspecified), (J440, Chronic obstructive pulmonary disease with acute lower respiratory in..."
7,6::274::293,systolic dysfunction,Diagnosis,0.8329,"[(I519, Heart disease, unspecified), (I5040, Unspecified combined systolic (congestive) and diastolic (congestive) heart failure), (I5020, Unspecified systolic (congestive) heart failure), (N522, Drug-induced erectile dysfunction), (F5221, Male e..."
9,10::223::264,acanthamoeba infection of skin and sinuses,Diagnosis,0.4519,"[(L089, Local infection of the skin and subcutaneous tissue, unspecified), (B6010, Acanthamebiasis, unspecified), (A311, Cutaneous mycobacterial infection), (B383, Cutaneous coccidioidomycosis), (L080, Pyoderma)]"
11,10::410::445,hepatomegaly and fluid surge spacing,Diagnosis,0.4627,"[(E8779, Other fluid overload), (E860, Dehydration), (E8770, Fluid overload, unspecified), (I313, Pericardial effusion (noninflammatory)), (J811, Chronic pulmonary edema)]"
12,10::456::478,setting of hyponatremia,Diagnosis,0.9990,"[(E871, Hypo-osmolality and hyponatremia), (I953, Hypotension of hemodialysis), (I952, Hypotension due to drugs), (E870, Hyperosmolality and hypernatremia), (J9602, Acute respiratory failure with hypercapnia)]"
15,14::323::330,ischemia,Diagnosis,0.8969,"[(G450, Vertebro-basilar artery syndrome), (N280, Ischemia and infarction of kidney), (H3582, Retinal ischemia), (I6782, Cerebral ischemia), (I248, Other forms of acute ischemic heart disease)]"


In [0]:
rxnorm_analysis[rxnorm_analysis.rxnorm_conf>0.4].head(20)

Out[184]:

,coords,chunk,entity,rxnorm_conf,rxnorm_opts
0,0::0::10,pentamidine,DrugChem,0.5925,"[(861601, Pentamidine Isethionate 300 MG Injection), (861597, Pentamidine Isethionate 50 MG/ML Inhalation Solution), (755627, Chloroquine 5 MG/ML Oral Solution), (855624, Dibromopropamidine isethionate 1 MG/ML Ophthalmic Solution), (1119497, chlo..."
1,0::37::47,pentamidine,DrugChem,0.5925,"[(861601, Pentamidine Isethionate 300 MG Injection), (861597, Pentamidine Isethionate 50 MG/ML Inhalation Solution), (755627, Chloroquine 5 MG/ML Oral Solution), (855624, Dibromopropamidine isethionate 1 MG/ML Ophthalmic Solution), (1119497, chlo..."
55,3::278::287,creatinine,DrugChem,0.9996,"[(849628, Creatinine 800 MG Oral Capsule), (252180, Urea 10 MG/ML Topical Lotion), (424168, Urea 30 MG/ML Topical Lotion), (251705, Urea 20 MG/ML Topical Lotion), (245052, Urea 200 MG/ML Oral Solution)]"
58,7::83::93,cholesterol,DrugChem,0.5609,"[(2104173, beta Sitosterol 35 MG Oral Tablet), (832876, phytosterol esters 500 MG Oral Capsule), (637208, phytosterol esters 650 MG Oral Capsule), (411217, Lecithin 228 MG Oral Capsule), (1737442, amphotericin B lipid complex 5 MG/ML Injection)]"
59,10::397::406,creatinine,DrugChem,0.9996,"[(849628, Creatinine 800 MG Oral Capsule), (252180, Urea 10 MG/ML Topical Lotion), (424168, Urea 30 MG/ML Topical Lotion), (251705, Urea 20 MG/ML Topical Lotion), (245052, Urea 200 MG/ML Oral Solution)]"
83,12::328::335,mestinon,DrugChem,0.4385,"[(2099309, moxetumomab pasudotox-tdfk 1 MG Injection), (886677, Clidinium bromide 2.5 MG Oral Capsule), (415693, Heparinoids 0.1 UNT/MG Topical Gel), (204558, Peptide Hydrolases 82 UNT/MG Topical Ointment), (1659998, ANTI-INHIBITOR COAGULANT COMP..."
84,12::372::379,mestinon,DrugChem,0.4385,"[(2099309, moxetumomab pasudotox-tdfk 1 MG Injection), (886677, Clidinium bromide 2.5 MG Oral Capsule), (415693, Heparinoids 0.1 UNT/MG Topical Gel), (204558, Peptide Hydrolases 82 UNT/MG Topical Ointment), (1659998, ANTI-INHIBITOR COAGULANT COMP..."
92,15::73::82,creatinine,DrugChem,0.9996,"[(849628, Creatinine 800 MG Oral Capsule), (252180, Urea 10 MG/ML Topical Lotion), (424168, Urea 30 MG/ML Topical Lotion), (251705, Urea 20 MG/ML Topical Lotion), (245052, Urea 200 MG/ML Oral Solution)]"


# SentenceEntityResolver (sbiobert) Models

- sbiobertresolve_icd10cm 
- sbiobertresolve_icd10cm_augmented
- sbiobertresolve_icd10pcs
- sbiobertresolve_snomed_findings (with clinical_findings concepts from CT version)
- sbiobertresolve_snomed_findings_int  (with clinical_findings concepts from INT version)
- sbiobertresolve_snomed_auxConcepts (with Morph Abnormality, Procedure, Substance, Physical Object, Body Structure concepts from CT version)
- sbiobertresolve_snomed_auxConcepts_int  (with Morph Abnormality, Procedure, Substance, Physical Object, Body Structure concepts from INT version)
- sbiobertresolve_rxnorm
- sbiobertresolve_rxcui
- sbiobertresolve_icdo
- sbiobertresolve_cpt
- sbiobertresolve_cpt_augmented

**!!! Warning**: ***If you get an error related to Java port not found 55, it is probably because that the Colab memory cannot handle the model and the Spark session died. In that case, try on a larger machine or restart the kernel at the top and then come back here and rerun. ***

## Sentence Entity Resolver - CPT

In [0]:
tokenizer = Tokenizer()\
      .setInputCols(["sentence"])\
      .setOutputCol("token")\

ner_converter_cpt = NerConverterInternal() \
      .setInputCols(["sentence", "token", "ner"]) \
      .setOutputCol("ner_chunk")\
      .setWhiteList(['TREATMENT','TEST'])\
      .setPreservePosition(False)

c2doc = Chunk2Doc()\
      .setInputCols("ner_chunk")\
      .setOutputCol("ner_chunk_doc") 

sbert_embedder = BertSentenceEmbeddings\
      .pretrained("sbiobert_base_cased_mli",'en','clinical/models')\
      .setInputCols(["ner_chunk_doc"])\
      .setOutputCol("sbert_embeddings")

cpt_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_cpt_augmented","en", "clinical/models") \
      .setInputCols(["ner_chunk", "sbert_embeddings"]) \
      .setOutputCol("sbert_cpt_code")\
      .setDistanceFunction("EUCLIDEAN")
  
sbert_resolver_pipeline = Pipeline(
    stages = [
        documentAssembler,
        sentenceDetectorDL,
        tokenizer,
        word_embeddings,
        clinical_ner,
        ner_converter_cpt,
        c2doc,
        sbert_embedder,
        cpt_resolver])


data_ner = spark.createDataFrame([['sample text']]).toDF("text")

sbert_models = sbert_resolver_pipeline.fit(data_ner)

sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[ | ][OK!]
sbiobertresolve_cpt_augmented download started this may take some time.
Approximate size to download 139.1 MB
[ | ][OK!]

In [0]:
sbert_light_pipeline_cpt = LightPipeline(sbert_models)

In [0]:
text = 'A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus (T2DM), one prior episode of HTG-induced pancreatitis three years prior to presentation, associated with an acute hepatitis, and obesity with a body mass index (BMI) of 33.5 kg/m2, presented with a one-week history of polyuria, polydipsia, poor appetite, and vomiting. Two weeks prior to presentation, she was treated with a five-day course of amoxicillin for a respiratory tract infection. She was on metformin, glipizide, and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG. She had been on dapagliflozin for six months at the time of presentation. Physical examination on presentation was significant for dry oral mucosa; significantly, her abdominal examination was benign with no tenderness, guarding, or rigidity. Pertinent laboratory findings on admission were: serum glucose 111 mg/dl, bicarbonate 18 mmol/l, anion gap 20, creatinine 0.4 mg/dL, triglycerides 508 mg/dL, total cholesterol 122 mg/dL, glycated hemoglobin (HbA1c) 10%, and venous pH 7.27. Serum lipase was normal at 43 U/L. Serum acetone levels could not be assessed as blood samples kept hemolyzing due to significant lipemia. The patient was initially admitted for starvation ketosis, as she reported poor oral intake for three days prior to admission. However, serum chemistry obtained six hours after presentation revealed her glucose was 186 mg/dL, the anion gap was still elevated at 21, serum bicarbonate was 16 mmol/L, triglyceride level peaked at 2050 mg/dL, and lipase was 52 U/L. The β-hydroxybutyrate level was obtained and found to be elevated at 5.29 mmol/L - the original sample was centrifuged and the chylomicron layer removed prior to analysis due to interference from turbidity caused by lipemia again. The patient was treated with an insulin drip for euDKA and HTG with a reduction in the anion gap to 13 and triglycerides to 1400 mg/dL, within 24 hours. Her euDKA was thought to be precipitated by her respiratory tract infection in the setting of SGLT2 inhibitor use. The patient was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night, 12 units of insulin lispro with meals, and metformin 1000 mg two times a day. It was determined that all SGLT2 inhibitors should be discontinued indefinitely. She had close follow-up with endocrinology post discharge.'

light_result = sbert_light_pipeline_cpt.annotate(text)

light_result.keys()

Out[102]: dict_keys(['document', 'ner_chunk', 'sbert_cpt_code', 'token', 'sbert_embeddings', 'ner', 'embeddings', 'ner_chunk_doc', 'sentence'])

In [0]:
light_result['sbert_cpt_code']

Out[104]: ['3008F',
 '80150',
 '83858',
 '82980',
 '1022259',
 '4145F',
 '80145',
 '1022259',
 '1014526',
 '76700',
 '1011445',
 '82374',
 '86039',
 '1011373',
 '84478',
 '82465',
 '83037',
 '83037',
 '82800',
 '83690',
 '83002',
 '82145',
 '38204',
 '86975',
 '1011445',
 '86039',
 '82374',
 '84478',
 '83690',
 '83002',
 '82953',
 '85347',
 '86337',
 '62117',
 '86039',
 '84478',
 '64823',
 '80435',
 '1011517',
 '83858',
 '83080']

In [0]:
df = pd.DataFrame(list(zip(light_result['ner_chunk'], light_result['sbert_cpt_code'])),
                  columns = ['Problem','cpt_code'])

df

Out[105]:

,Problem,cpt_code
0,BMI,3008F
1,amoxicillin,80150
2,metformin,83858
3,glipizide,82980
4,dapagliflozin,1022259
5,atorvastatin,4145F
6,gemfibrozil,80145
7,dapagliflozin,1022259
8,Physical examination,1014526
9,her abdominal examination,76700


In [0]:
df = get_codes (sbert_light_pipeline_cpt, text, 'sbert_cpt_code')

df

Out[106]:

,chunks,begin,end,sbert_cpt_code,resolutions
0,BMI,316,318,3008F,"Body Mass Index (BMI), documented (PV):::Weight monitoring:::Weight monitoring:::Weighing:::Glob..."
1,amoxicillin,499,509,80150,Amikacin:::Gentamicin:::Antibiotic coverage:::Antibiotic coverage:::Antibiotic coverage:::Antibi...
2,metformin,557,565,83858,Methsuximide:::Methadone:::Methadone:::Metabolic function test:::Metyrapone panel:::Insulin:::Fa...
3,glipizide,568,576,82980,Glutethimide:::Digoxin:::Glucagon:::Lamotrigine:::Carbamazepine:::Lacosamide:::Tiagabine:::Gluca...
4,dapagliflozin,583,595,1022259,Digoxin:::Zonisamide:::Tiagabine:::Doxepin:::Dilantin measurement:::Dilantin measurement:::Lamot...
5,atorvastatin,610,621,4145F,Antihypertensive therapy (procedure):::Angiotensin converting enzyme inhibitor therapy:::Angiote...
6,gemfibrozil,627,637,80145,Adalimumab:::Vedolizumab:::Biological response modifier therapy:::Drug therapy:::Drug therapy:::...
7,dapagliflozin,664,676,1022259,Digoxin:::Zonisamide:::Tiagabine:::Doxepin:::Dilantin measurement:::Dilantin measurement:::Lamot...
8,Physical examination,722,741,1014526,Physical Examination:::Physical assessment:::Monitors physical parameters:::Monitors physical pa...
9,her abdominal examination,811,835,76700,US abdominal scan:::US abdominal scan:::Endoscopy of abdomen:::Endoscopy of abdomen:::Endoscopy ...


In [0]:
df = get_codes (sbert_light_pipeline_cpt, 'The patient needs to have a coronary artery bypass but doctor suggests an abdomen CT at first.', 'sbert_cpt_code')

df

Out[107]:

,chunks,begin,end,sbert_cpt_code,resolutions
0,a coronary artery bypass,26,49,1006216,"Arterial Grafting for Coronary Artery Bypass:::Coronary artery bypass, using arterial graft(s)::..."
1,an abdomen CT,71,83,1010526,"Computed tomography, abdomen:::US abdominal scan:::US abdominal scan:::Radiologic examination, a..."


In [0]:
df['resolutions'][0]

Out[108]: 'Arterial Grafting for Coronary Artery Bypass:::Coronary artery bypass, using arterial graft(s):::Coronary artery graft placement:::Coronary artery graft placement:::Coronary artery graft placement:::Aortic bifurcation bypass graft:::Aortic bifurcation bypass graft:::Procedure on coronary arteries:::Aortoiliac vascular bypass:::Aortoiliac vascular bypass:::Aortoiliac vascular bypass:::Aortoiliac vascular bypass:::Aortoiliac vascular bypass:::Arterial bypass graft:::Arterial bypass graft:::Arterial bypass graft:::Arterial bypass graft:::Coronary artery reconstruction:::Coronary artery reconstruction:::Reimplantation of coronary artery:::Reimplantation of coronary artery'

In [0]:
df['resolutions'][1]

Out[109]: 'Computed tomography, abdomen:::US abdominal scan:::US abdominal scan:::Radiologic examination, abdomen:::Radiologic examination, abdomen:::Computed tomography, abdomen; with contrast material(s):::Magnetic resonance imaging of abdomen:::Magnetic resonance imaging of abdomen:::Computed tomography of abdominal vascular structures:::Procedure on abdomen:::Procedure on abdomen:::Endoscopy of abdomen:::Endoscopy of abdomen:::Endoscopy of abdomen:::Endoscopy of abdomen:::Endoscopy of abdomen:::Endoscopy of abdomen:::Endoscopy of abdomen:::Endoscopy of abdomen:::Endoscopy of abdomen:::Endoscopy of abdomen'

## Sentence Entity Resolver - RxNorm

In [0]:
documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("ner_chunk")

sbert_embedder = BertSentenceEmbeddings\
      .pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("sbert_embeddings")
    
rxnorm_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_rxnorm","en", "clinical/models") \
      .setInputCols(["ner_chunk", "sbert_embeddings"]) \
      .setOutputCol("rxnorm_code")\
      .setDistanceFunction("EUCLIDEAN")

rxnorm_pipelineModel = PipelineModel(
    stages = [
        documentAssembler,
        sbert_embedder,
        rxnorm_resolver])

rxnorm_lp = LightPipeline(rxnorm_pipelineModel)


sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[ | ][OK!]
sbiobertresolve_rxnorm download started this may take some time.
Approximate size to download 810.7 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][OK!]

In [0]:
import pandas as pd

pd.set_option('display.max_colwidth', 0)

def get_rxnorm_codes (lp, text, vocab='rxnorm_code'):
    
    full_light_result = lp.fullAnnotate(text)

    chunks = []
    codes = []
    begin = []
    end = []
    resolutions=[]
    entities=[]
    all_distances =[]
    all_codes=[]
    
    for chunk, code in zip(full_light_result[0]['ner_chunk'], full_light_result[0][vocab]):
            
        begin.append(chunk.begin)
        end.append(chunk.end)
        chunks.append(chunk.result)
        entities.append(chunk.metadata['entity']) 
        codes.append(code.result) 
        all_codes.append(code.metadata['all_k_results'].split(':::'))
        resolutions.append(code.metadata['all_k_resolutions'].split(':::'))
        all_distances.append(code.metadata['all_k_distances'].split(':::'))

    df = pd.DataFrame({'chunks':chunks, 'begin': begin, 'end':end, 'entity':entities,
                    'code':codes,'all_codes':all_codes, 
                    'resolutions':resolutions, 'all_distances':all_distances})

    
    return df



In [0]:
text = 'metformin 100 mg'

get_rxnorm_codes (rxnorm_lp, text, vocab='rxnorm_code')

Out[121]:

,chunks,begin,end,entity,code,all_codes,resolutions,all_distances
0,metformin 100 mg,0,15,None,406081,"[406081, 576612, 403968, 861024, 404727, 334738, 332848, 861026, 333262, 439563, 450523, 1744000, 484793, 402346, 1726496, 316350, 858858, 336846, 316844, 1946837, 451225, 328507, 437723, 385601, 315677]","[metformin 100 mg/ml, metformin 100 mg/ml [riomet], metformin 100 mg/ml oral solution, metformin hydrochloride 100 mg/ml, metformin 100 mg/ml oral solution [riomet], fenofibrate 100 mg, ciprofibrate 100 mg, metformin hydrochloride 100 mg/ml [riomet], rutin 100 mg, fendiline 100 mg, perazine 100 mg, emtricitabine 100 mg, solifenacin 100 mg, miglustat 100 mg, azacitidine 100 mg, niacin 100 mg, carnosine 100 mg, trimebutine 100 mg, torsemide 100 mg, abemaciclib 100 mg, pyrantel 100 mg, rimantadine 100 mg, azintamide 100 mg, mebeverine 100 mg, cimetidine 100 mg]","[3.8387, 4.8433, 5.6433, 6.3835, 6.4686, 6.5293, 6.5728, 6.9061, 6.9297, 6.9511, 7.0943, 7.1014, 7.1202, 7.1323, 7.1327, 7.1425, 7.2076, 7.2118, 7.2518, 7.2531, 7.2533, 7.2761, 7.2802, 7.2915, 7.3286]"


In [0]:
text = 'aspirin 10 meq/ 5 ml oral sol'

get_rxnorm_codes (rxnorm_lp, text, 'rxnorm_code')

Out[122]:

,chunks,begin,end,entity,code,all_codes,resolutions,all_distances
0,aspirin 10 meq/ 5 ml oral sol,0,28,None,104920,"[104920, 410146, 636574, 729048, 422031, 205247, 251102, 246529, 544468, 755941, 251830, 1117402, 422305, 241082, 245328, 604932, 247061, 604936, 2057152, 206399, 830193, 2057158, 252942, 794260, 428592]","[aspirin 500 mg / papaveretum 10 mg oral solution, cromoglicic acid 10 mg oral capsule, guaifenesin 10 mg/ml oral solution, pseudoephedrine tannate 10 mg/ml oral suspension, silymarin 10 mg/ml oral suspension, niacin 10 mg/ml oral solution, nimesulide 10 mg/ml oral suspension, midodrine 10 mg/ml oral solution, hypromellose 10 mg/ml oral solution, teferrol 10 mg/ml oral solution, mebeverine 10 mg/ml oral solution, guaifenesin 10 mg/ml oral solution [liqufruta], mebeverine 10 mg/ml oral suspension, methadyl acetate 10 mg/ml oral solution, cascara sagrada 10 mg/ml oral solution, ubidecarenone 10 mg/ml oral solution, periciazine 10 mg/ml oral solution, ubidecarenone 10 mg/ml oral solution [liquid co-q10], telmisartan 10 mg/ml oral solution, methadyl acetate 10 mg/ml oral solution [orlaam], opium tincture 10 mg/ml oral solution, telmisartan 10 mg/ml oral solution [semintra], bismuth subsalicylate 10 mg/ml oral suspension, ferrous sulfate 10 mg/ml oral solution, melitracen 10 mg / periciazine 0.5 mg oral tablet]","[6.3399, 7.3543, 7.4186, 7.4942, 7.5560, 7.6451, 7.6566, 7.7084, 7.7125, 7.7731, 7.8107, 7.8234, 7.8349, 7.8351, 7.8493, 7.8585, 7.8976, 7.9211, 7.9405, 7.9455, 7.9622, 7.9711, 8.0290, 8.0389, 8.0513]"


##RxNorm with DrugNormalizer

In [0]:
documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("ner_chunk_v0")

drug_normalizer = DrugNormalizer() \
      .setInputCols("ner_chunk_v0") \
      .setOutputCol("ner_chunk") \
      .setPolicy('all')

rxnorm_pipelineModel2 = PipelineModel(
    stages = [
        documentAssembler,
        drug_normalizer,
        sbert_embedder,
        rxnorm_resolver])

rxnorm_lp2 = LightPipeline(rxnorm_pipelineModel2)

In [0]:
text = 'aspirin 10 meq/ 5 ml oral sol'

get_rxnorm_codes (rxnorm_lp2, text, vocab='rxnorm_code')

CPU times: user 13.7 ms, sys: 2.78 ms, total: 16.5 ms
Wall time: 541 ms
Out[126]:

,chunks,begin,end,entity,code,all_codes,resolutions,all_distances
0,aspirin 2 meq/ml oral solution,0,29,None,688214,"[688214, 342917, 688213, 904947, 343786, 1192949, 755937, 756028, 104072, 343790, 246229, 755924, 309427, 755920, 756064, 756078, 607610, 756171, 1115998, 248847, 243387, 250944, 577157, 108870, 421803]","[aspirin 2.5 mg/ml oral solution, aspirin 2.2 mg/ml, aspirin 2.5 mg/ml, aspirin 2.7 mg/ml, aspirin 2.75 mg/ml, aspirin 2.71 mg/ml, periciazine 2 mg/ml oral solution, fenspiride 2 mg/ml oral solution, cimetidine 2 mg/ml oral solution, aspirin 2.12 mg/ml, thioridazine 2 mg/ml oral solution, carbetapentane 2 mg/ml oral solution, codeine 2 mg/ml oral solution, butamirate 2 mg/ml oral solution, oxeladin 2 mg/ml oral solution, pipazethate 2 mg/ml oral solution, memantine 2 mg/ml oral solution [namenda], selegiline 2 mg/ml oral solution, ephedrine sulfate 2.2 mg/ml oral solution, homatropine 2 mg/ml oral solution, docusate 2 mg/ml oral suspension, piracetam 2 mg/ml oral solution, memantine 2 mg/ml oral solution, theophylline 2 mg/ml oral solution, codeine 2 mg/ml oral suspension]","[4.3443, 5.5582, 6.3624, 6.3805, 7.0121, 7.1228, 7.1609, 7.4941, 7.5412, 7.7995, 7.8994, 7.9552, 7.9593, 7.9684, 7.9984, 8.0101, 8.0192, 8.0963, 8.1114, 8.1328, 8.1864, 8.1967, 8.2164, 8.2222, 8.2508]"


## Sentence Entity Resolver - SNOMED

In [0]:
documentAssembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

sentenceDetector = SentenceDetector()\
  .setInputCols(["document"])\
  .setOutputCol("sentence")

tokenizer = Tokenizer()\
  .setInputCols(["sentence"])\
  .setOutputCol("token")\

word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
  .setInputCols(["sentence", "token"])\
  .setOutputCol("embeddings")

clinical_ner = MedicalNerModel.pretrained("ner_clinical", "en", "clinical/models") \
  .setInputCols(["sentence", "token", "embeddings"]) \
  .setOutputCol("ner")

ner_converter = NerConverter() \
  .setInputCols(["sentence", "token", "ner"]) \
  .setOutputCol("ner_chunk")\
  .setWhiteList(['PROBLEM'])

c2doc = Chunk2Doc().setInputCols("ner_chunk").setOutputCol("ner_chunk_doc") 

bert_embeddings = BertSentenceEmbeddings.pretrained("sbiobert_base_cased_mli", 'en', 'clinical/models')\
  .setInputCols(["ner_chunk_doc"])\
  .setOutputCol("bert_embeddings")

snomed_resolution = SentenceEntityResolverModel.pretrained("sbiobertresolve_snomed_findings", "en", "clinical/models") \
  .setInputCols(["ner_chunk", "bert_embeddings"]) \
  .setOutputCol("snomed_code")

pipeline_snomed = Pipeline(
    stages = [
    documentAssembler,
    sentenceDetector,
    tokenizer,
    word_embeddings,
    clinical_ner,
    ner_converter,
    c2doc,
    bert_embeddings,
    snomed_resolution
  ])


embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
ner_clinical download started this may take some time.
Approximate size to download 13.9 MB
[ | ][OK!]
sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[ | ][OK!]
sbiobertresolve_snomed_findings download started this may take some time.
Approximate size to download 1.4 GB
[ | ][OK!]

In [0]:
clinical_note = (
    'A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years '
    'prior to presentation and subsequent type two diabetes mellitus (T2DM), one prior '
    'episode of HTG-induced pancreatitis three years prior to presentation, associated '
    'with an acute hepatitis, and obesity with a body mass index (BMI) of 33.5 kg/m2, '
    'presented with a one-week history of polyuria, polydipsia, poor appetite, and vomiting. '
    'Two weeks prior to presentation, she was treated with a five-day course of amoxicillin '
    'for a respiratory tract infection. She was on metformin, glipizide, and dapagliflozin '
    'for T2DM and atorvastatin and gemfibrozil for HTG. She had been on dapagliflozin for six months '
    'at the time of presentation. Physical examination on presentation was significant for dry oral mucosa; '
    'significantly, her abdominal examination was benign with no tenderness, guarding, or rigidity. Pertinent '
    'laboratory findings on admission were: serum glucose 111 mg/dl, bicarbonate 18 mmol/l, anion gap 20, '
    'creatinine 0.4 mg/dL, triglycerides 508 mg/dL, total cholesterol 122 mg/dL, glycated hemoglobin (HbA1c) '
    '10%, and venous pH 7.27. Serum lipase was normal at 43 U/L. Serum acetone levels could not be assessed '
    'as blood samples kept hemolyzing due to significant lipemia. The patient was initially admitted for '
    'starvation ketosis, as she reported poor oral intake for three days prior to admission. However, '
    'serum chemistry obtained six hours after presentation revealed her glucose was 186 mg/dL, the anion gap '
    'was still elevated at 21, serum bicarbonate was 16 mmol/L, triglyceride level peaked at 2050 mg/dL, and '
    'lipase was 52 U/L. The β-hydroxybutyrate level was obtained and found to be elevated at 5.29 mmol/L - '
    'the original sample was centrifuged and the chylomicron layer removed prior to analysis due to '
    'interference from turbidity caused by lipemia again. The patient was treated with an insulin drip '
    'for euDKA and HTG with a reduction in the anion gap to 13 and triglycerides to 1400 mg/dL, within '
    '24 hours. Her euDKA was thought to be precipitated by her respiratory tract infection in the setting '
    'of SGLT2 inhibitor use. The patient was seen by the endocrinology service and she was discharged on '
    '40 units of insulin glargine at night, 12 units of insulin lispro with meals, and metformin 1000 mg '
    'two times a day. It was determined that all SGLT2 inhibitors should be discontinued indefinitely. She '
    'had close follow-up with endocrinology post discharge.'
)

data_ner = spark.createDataFrame([[clinical_note]]).toDF("text")

snomed_output = pipeline_snomed.fit(data_ner).transform(data_ner)


In [0]:
df = snomed_output.select(F.explode(F.arrays_zip("ner_chunk.result","ner_chunk.metadata","snomed_code.result","snomed_code.metadata"))\
                          .alias("snomed_result")) \
                  .select(F.expr("snomed_result['0']").alias("chunk"),
                          F.expr("snomed_result['1'].entity").alias("entity"),
                          F.expr("snomed_result['3'].all_k_resolutions").alias("target_text"),
                          F.expr("snomed_result['2']").alias("snomed_code"),
                          F.expr("snomed_result['3'].confidence").alias("distance")).toPandas()

df

Out[206]:

,chunk,entity,target_text,snomed_code,distance
0,gestational diabetes mellitus,PROBLEM,gestational diabetes mellitus:::gestational diabetes mellitus:::gestational diabetes mellitus:::gestational diabetes mellitus:::maternal gestational diabetes mellitus:::maternal diabetes mellitus:::postpartum gestational diabetes mellitus:::pre-e...,393568003,0.2475
1,subsequent type two diabetes mellitus,PROBLEM,pre-existing type 2 diabetes mellitus:::disorder associated with type 2 diabetes mellitus:::type 2 diabetes mellitus:::secondary diabetes mellitus:::multiple complications due to type 2 diabetes mellitus:::complication due to diabetes mellitus ty...,199230006,0.1653
2,T2DM,PROBLEM,type 2 diabetes mellitus:::disorder associated with type 2 diabetes mellitus:::pre-existing type 2 diabetes mellitus:::type ii diabetes mellitus:::diabetes mellitus:::diabetes mellitus:::diabetes mellitus:::diabetes mellitus:::secondary diabetes ...,44054006,0.1989
3,HTG-induced pancreatitis,PROBLEM,alcohol-induced pancreatitis:::drug-induced acute pancreatitis:::alcohol-induced acute pancreatitis:::hemorrhagic pancreatitis:::alcohol-induced chronic pancreatitis:::alcohol-induced chronic pancreatitis:::pancreatitis:::pancreatitis:::pancreati...,445507008,0.1443
4,an acute hepatitis,PROBLEM,acute hepatitis a:::acute hepatitis:::acute infectious hepatitis:::acute hepatitis e:::acute hepatitis e:::acute viral hepatitis:::acute focal hepatitis:::toxic liver disease with acute hepatitis:::acute fulminating type a viral hepatitis:::fulmi...,25102003,0.3444
5,obesity,PROBLEM,obesity:::obesity:::obesity:::abdominal obesity:::generalized obesity:::obese:::central obesity:::morbid obesity:::morbid obesity:::morbid obesity:::obese abdomen:::obese build:::severe obesity:::o/e - obese:::o/e - obese:::o/e - obese:::o/e - ob...,414916001,0.3193
6,a body mass index,PROBLEM,finding of body mass index:::mass of body region:::mass of body structure:::a mass:::mass of upper limb:::head and neck mass:::mass of trunk:::mass of head:::finding of body composition:::finding of head circumference:::body positions:::preoccupa...,301331008,0.6762
7,polyuria,PROBLEM,polyuria:::polyuria:::polyuria:::polyuric state:::hematuria:::hematuria:::hematuria:::micturition frequency and polyuria:::uricosuria:::pollakiuria:::saccharopinuria:::saccharopinuria:::oligouria:::dysuria:::dysuria:::hematuria syndrome:::cystinu...,28442001,0.3330
8,polydipsia,PROBLEM,polydipsia:::polydipsia:::psychogenic polydipsia:::psychogenic polydipsia:::primary polydipsia:::polyhydramnios:::polyhydramnios:::polyhydramnios:::parasomnia:::polyalgia:::polyalgia:::polyphagia:::polyphagia:::biphasic stridor:::oscillopsia:::pa...,161847005,0.4972
9,poor appetite,PROBLEM,poor appetite:::lack of appetite:::lack of appetite:::poor feeding:::loss of appetite:::loss of appetite:::bad taste in mouth:::bad taste in mouth:::bad taste in mouth:::bad taste in mouth:::decreased sense of taste:::poor fluid intake:::bad brea...,64379006,0.9835


### with SNOMED INT

In [0]:

snomed_resolution_int = SentenceEntityResolverModel.pretrained("sbiobertresolve_snomed_findings_int", "en", "clinical/models") \
  .setInputCols(["ner_chunk", "bert_embeddings"]) \
  .setOutputCol("snomed_code_int")

pipeline_snomed_int = Pipeline(
    stages = [
    documentAssembler,
    sentenceDetector,
    tokenizer,
    word_embeddings,
    clinical_ner,
    ner_converter,
    c2doc,
    bert_embeddings,
    snomed_resolution_int
  ])

snomed_output_int = pipeline_snomed_int.fit(data_ner).transform(data_ner)


sbiobertresolve_snomed_findings_int download started this may take some time.
Approximate size to download 517.4 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][OK!]

In [0]:
snomed_output_int.select(F.explode(F.arrays_zip("ner_chunk.result","ner_chunk.metadata","snomed_code_int.result","snomed_code_int.metadata"))\
                         .alias("snomed_result")) \
                  .select(F.expr("snomed_result['0']").alias("chunk"),
                          F.expr("snomed_result['1'].entity").alias("entity"),
                          F.expr("snomed_result['3'].all_k_resolutions").alias("target_text"),
                          F.expr("snomed_result['2']").alias("snomed_code"),
                          F.expr("snomed_result['3'].confidence").alias("distance")).show(truncate = 100)

+-------------------------------------+-------+----------------------------------------------------------------------------------------------------+-----------+--------+
 chunk| entity| target_text|snomed_code|distance|
+-------------------------------------+-------+----------------------------------------------------------------------------------------------------+-----------+--------+
 gestational diabetes mellitus|PROBLEM|ureterocele - acquired:::gestational diabetes mellitus:::glomus tumour:::faeces colour finding:::...| 197822006| 0.3291|
subsequent type two diabetes mellitus|PROBLEM|pre-existing type 2 diabetes mellitus:::disorder due to type 2 diabetes mellitus:::type 2 diabete...| 199230006| 0.1294|
 T2DM|PROBLEM|type 2 diabetes mellitus:::pre-existing type 2 diabetes mellitus:::type i diabetes mellitus with ...| 44054006| 0.1900|
 HTG-induced pancreatitis|PROBLEM|alcohol-induced pancreatitis:::drug-induced acute pancreatitis:::alcohol-induced acute pancreatit...| 445507008| 0.1257|
 an acute hepatitis|PROBLEM|acute hepatitis:::acute infectious hepatitis:::hepatitis d superinfection of hepatitis b carrier:...| 37871000| 0.4490|
 obesity|PROBLEM|morbid obesity:::fat pad syndrome:::disorder of metabolism nos:::maternal obesity syndrome:::obes...| 238136002| 0.3211|
 a body mass index|PROBLEM|respiratory symptoms nos:::finding of body mass index:::mass of body region:::o/e - dead - condit...| 139218003| 0.3884|
 polyuria|PROBLEM|o/e - hypothermia:::polyuria:::polyuric state:::micturition frequency and polyuria:::saccharopinu...| 141496003| 0.4995|
 polydipsia|PROBLEM|born after multiple pregnancy:::psychogenic polydipsia:::polyhydramnios:::oligohydramnios:::feto-...| 157053003| 0.9948|
 poor appetite|PROBLEM|observation of skin texture:::poor nutrition:::feeding poor:::loss of appetite:::bad taste in mou...| 271814001| 0.2175|
 vomiting|PROBLEM|vomiting:::no vomiting:::c/o - vomiting:::intermittent vomiting:::nausea nos:::vomit odour nos:::...| 422400008| 0.3263|
 a respiratory tract infection|PROBLEM|respiratory tract infection:::bacterial respiratory infection:::upper respiratory infection:::res...| 275498002| 0.6345|
 T2DM|PROBLEM|type 2 diabetes mellitus:::pre-existing type 2 diabetes mellitus:::type i diabetes mellitus with ...| 44054006| 0.1900|
 HTG|PROBLEM|thomas' sign:::10g monofilament sensation l foot abnormal:::gerbich type:::geotrichosis:::[x]muco...| 68193004| 0.1006|
 dry oral mucosa|PROBLEM|clothing dishevelled:::dry skin:::dry tongue:::dry hair:::salivation excess symp.:::c/o: dry skin...| 248163004| 0.1447|
 tenderness|PROBLEM|tenderness:::adverse reaction to non-steroidal anti-inflammatory drug:::o/e - tenderness:::o/e - ...| 247348008| 0.4982|
 guarding|PROBLEM|abnormal preening behavior:::no problems with thinking:::spiritual problem:::aids:::reflecting:::...| 471295005| 0.1598|
 rigidity|PROBLEM|rigid character:::rigid chest:::decorticate rigidity:::muscle rigidity:::emotional stability:::di...| 69193009| 0.7795|
 hemolyzing|PROBLEM|hemolysis:::sample hemolyzed:::hemolysin factor:::extravascular hemolysis:::mechanical hemolytic ...| 73320003| 0.4946|
 significant lipemia|PROBLEM|lipids abnormal:::serum lipids high:::serum lipids borderline raised:::high lipase level in serum...| 166818002| 0.1612|
+-------------------------------------+-------+----------------------------------------------------------------------------------------------------+-----------+--------+
only showing top 20 rows

In [0]:

snomed_resolution_int = SentenceEntityResolverModel.pretrained("biobertresolve_snomed_findings", "en", "clinical/models") \
  .setInputCols(["ner_chunk", "bert_embeddings"]) \
  .setOutputCol("snomed_code_int")


biobertresolve_snomed_findings download started this may take some time.
Approximate size to download 597.1 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][OK!]

End of Notebook